# From SAV to CSV

<b>THIS NOTEBOOK</b><br>
In this notebook - after some exploration and trying to make sense of the data - I turn a SPSS-file into several CSVs. 
<ol>
    <li> a csv from which NaNs are deleted </li>
    <li> a csv from which some NaNs are deleted but others are filled </li>
    <li> a csv in which all NaNs are filled</li>
    </ol>

<b>THE DATA </b><br>
More information about this specific survey: https://www.europeansocialsurvey.org/data/themes.html?t=climatech <br>
Link to the data: https://www.europeansocialsurvey.org/download.html?file=ESS8e02_2&y=2016 <br>
Overview of different editions and topics: http://nesstar.ess.nsd.uib.no/webview/index.jsp?v=2&submode=abstract&study=http%3A%2F%2F129.177.90.83%3A80%2Fobj%2FfStudy%2FESS9e03.1&mode=documentation&top=yes <br>

## Preparations 

The data is downloadable in several formats: SPSS, SAS and STATA. I've chosen to work with the SPSS-file, which is readable through the pyreadstat package.

Documentation on pyreadstat: https://ofajardo.github.io/pyreadstat_documentation/_build/html/index.html

In [1]:
# load packages
import pyreadstat 
import pandas as pd
import numpy as np

In [2]:
#load dataset: give it some time!
SPSS = pyreadstat.read_sav('./data/ESS8e02_2.sav') #adjust to your file location and check file name

### Getting to know the data

In [3]:
#find out how data is stored in the spss-file
print(type(SPSS)) 
print(type(SPSS[0]))
print(type(SPSS[1]))

<class 'tuple'>
<class 'pandas.core.frame.DataFrame'>
<class 'pyreadstat._readstat_parser.metadata_container'>


Check out the dataframe first. Look at it's shape, first few rows and data-types.

In [4]:
climate = SPSS[0]
print(climate.shape)

(44387, 535)


In [5]:
climate.head()

,name,essround,edition,proddate,idno,cntry,nwspol,netusoft,netustm,ppltrst,...,inwdde,inwmme,inwyye,inwehh,inwemm,inwtm,dweight,pspwght,pweight,anweight
0,ESS8e02_2,8.0,2.2,10.12.2020,1.0,AT,120.0,4.0,180.0,8.0,...,5.0,12.0,2016.0,8.0,33.0,50.0,0.611677,1.178496,0.370393,0.436506
1,ESS8e02_2,8.0,2.2,10.12.2020,2.0,AT,120.0,5.0,120.0,6.0,...,25.0,11.0,2016.0,11.0,10.0,86.0,1.223354,0.899472,0.370393,0.333158
2,ESS8e02_2,8.0,2.2,10.12.2020,4.0,AT,30.0,2.0,NaN,5.0,...,22.0,11.0,2016.0,18.0,0.0,38.0,0.389058,0.315753,0.370393,0.116953
3,ESS8e02_2,8.0,2.2,10.12.2020,6.0,AT,30.0,5.0,120.0,6.0,...,11.0,10.0,2016.0,19.0,59.0,46.0,0.642594,0.472467,0.370393,0.174999
4,ESS8e02_2,8.0,2.2,10.12.2020,10.0,AT,30.0,5.0,180.0,5.0,...,1.0,12.0,2016.0,11.0,49.0,70.0,3.432402,2.246706,0.370393,0.832164


In [6]:
climate.dtypes.sort_values(ascending=False)

name         object
region       object
cntbrthc     object
ctzshipc     object
fbrncntb     object
             ...   
wkdcorga    float64
iorgact     float64
wkhct       float64
icwhct      float64
yrbrn2      float64
Length: 535, dtype: object

The data exists of objects and floats. Objects could still be several things, so let's check them out in more detail. They turn out to be all strings:

In [7]:
#check out the non-float types (all objects)
print(type(climate["name"][0]))
print(type(climate["region"][0]))
print(type(climate["cntbrthc"][0]))
print(type(climate["ctzshipc"][0]))
print(type(climate["fbrncntb"][0]))
print(type(climate["mbrncntb"][0]))
print(type(climate["lnghom1"][0]))
print(type(climate["lnghom2"][200])) #0 = NaN (float). Choose other row
print(type(climate["edition"][0]))
print(type(climate["proddate"][0]))
print(type(climate["cntry"][0]))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


<b> THE METADATA </b>

Checkout the metadata (SPSS[1]). According to the documentation, all sorts of information could be stored here. Like:
column_names; column_labels; column_names_to_labels; file_encoding; number_columns; number_rows etc..

Of special interest<br>
<ul>
    <li>column_names_to_labels: this is a dictionary that tells you which questions/data corresponds with which column abbreviation. For example, 'edlvdpt' stands for the question: 'Highest level of education, Portugal'.</li>
    <li>column_labels might also come in handy, this is a list of the questions/data.</li>
</ul>

pyreadstat documentation: https://ofajardo.github.io/pyreadstat_documentation/_build/html/index.html

In [8]:
metadata = SPSS[1]

In [159]:
type(metadata)

pyreadstat._readstat_parser.metadata_container

In [9]:
metadata.column_names_to_labels

{'name': 'Title of dataset',
 'essround': 'ESS round',
 'edition': 'Edition',
 'proddate': 'Production date',
 'idno': "Respondent's identification number",
 'cntry': 'Country',
 'nwspol': 'News about politics and current affairs, watching, reading or listening, in minutes',
 'netusoft': 'Internet use, how often',
 'netustm': 'Internet use, how much time on typical day, in minutes',
 'ppltrst': "Most people can be trusted or you can't be too careful",
 'pplfair': 'Most people try to take advantage of you, or try to be fair',
 'pplhlp': 'Most of the time people helpful or mostly looking out for themselves',
 'polintr': 'How interested in politics',
 'psppsgva': 'Political system allows people to have a say in what government does',
 'actrolga': 'Able to take active role in political group',
 'psppipla': 'Political system allows people to have influence on politics',
 'cptppola': 'Confident in own ability to participate in politics',
 'trstprl': "Trust in country's parliament",
 'trstlgl

In [10]:
metadata.column_labels

['Title of dataset',
 'ESS round',
 'Edition',
 'Production date',
 "Respondent's identification number",
 'Country',
 'News about politics and current affairs, watching, reading or listening, in minutes',
 'Internet use, how often',
 'Internet use, how much time on typical day, in minutes',
 "Most people can be trusted or you can't be too careful",
 'Most people try to take advantage of you, or try to be fair',
 'Most of the time people helpful or mostly looking out for themselves',
 'How interested in politics',
 'Political system allows people to have a say in what government does',
 'Able to take active role in political group',
 'Political system allows people to have influence on politics',
 'Confident in own ability to participate in politics',
 "Trust in country's parliament",
 'Trust in the legal system',
 'Trust in the police',
 'Trust in politicians',
 'Trust in political parties',
 'Trust in the European Parliament',
 'Trust in the United Nations',
 'Voted last national ele

In [11]:
print(metadata.file_encoding)
print(metadata.number_columns) # same number of columns as indicated by climate.shape
print(metadata.number_rows) # same number of rows as indicated by climate.shape

UTF-8
535
44387


In [12]:
print(metadata.missing_user_values)
print(metadata.missing_ranges)
metadata.variable_measure

{}
{}


{'name': 'nominal',
 'essround': 'nominal',
 'edition': 'nominal',
 'proddate': 'nominal',
 'idno': 'nominal',
 'cntry': 'nominal',
 'nwspol': 'scale',
 'netusoft': 'nominal',
 'netustm': 'scale',
 'ppltrst': 'nominal',
 'pplfair': 'nominal',
 'pplhlp': 'nominal',
 'polintr': 'nominal',
 'psppsgva': 'nominal',
 'actrolga': 'nominal',
 'psppipla': 'nominal',
 'cptppola': 'nominal',
 'trstprl': 'nominal',
 'trstlgl': 'nominal',
 'trstplc': 'nominal',
 'trstplt': 'nominal',
 'trstprt': 'nominal',
 'trstep': 'nominal',
 'trstun': 'nominal',
 'vote': 'nominal',
 'prtvtbat': 'nominal',
 'prtvtcbe': 'nominal',
 'prtvtfch': 'nominal',
 'prtvtdcz': 'nominal',
 'prtvede1': 'nominal',
 'prtvede2': 'nominal',
 'prtvtfee': 'nominal',
 'prtvtdes': 'nominal',
 'prtvtdfi': 'nominal',
 'prtvtcfr': 'nominal',
 'prtvtbgb': 'nominal',
 'prtvtehu': 'nominal',
 'prtvtbie': 'nominal',
 'prtvtcil': 'nominal',
 'prtvtbis': 'nominal',
 'prtvtbit': 'nominal',
 'prtvblt1': 'nominal',
 'prtvblt2': 'nominal',
 'prt

In [13]:
#create dataframe with shorthands and type of data (nom_or_scale)
overview = pd.DataFrame.from_dict(metadata.variable_measure, orient='index', dtype=None, columns=["nom_or_scale"])
overview.reset_index(level=0, inplace=True) 
overview.rename(columns={overview.columns[0]: 'shorthand'}, inplace=True)
overview.head()

,shorthand,nom_or_scale
0,name,nominal
1,essround,nominal
2,edition,nominal
3,proddate,nominal
4,idno,nominal


In [14]:
# add the full questions to the dataframe
to_append = pd.Series(metadata.column_labels)

overview = pd.concat([overview, to_append], axis=1, ignore_index=False)
overview.rename(columns={overview.columns[2]: 'full_text'}, inplace=True)
# pd.DataFrame.join([overview, to_append], axis=1)
overview.head()

,shorthand,nom_or_scale,full_text
0,name,nominal,Title of dataset
1,essround,nominal,ESS round
2,edition,nominal,Edition
3,proddate,nominal,Production date
4,idno,nominal,Respondent's identification number


In [15]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
overview

,shorthand,nom_or_scale,full_text
0,name,nominal,Title of dataset
1,essround,nominal,ESS round
2,edition,nominal,Edition
3,proddate,nominal,Production date
4,idno,nominal,Respondent's identification number
5,cntry,nominal,Country
6,nwspol,scale,"News about politics and current affairs, watching, reading or listening, in minutes"
7,netusoft,nominal,"Internet use, how often"
8,netustm,scale,"Internet use, how much time on typical day, in minutes"
9,ppltrst,nominal,Most people can be trusted or you can't be too careful


This table might come in very handy, will store it in a CSV.

In [163]:
overview.to_csv('./data/metadata.csv', index=False)

### NaN-handling
<b>GETTING TO KNOW THE DATA BETTER</b><br>
Columns that are mainly empty are colored <span style='color:red'>red</span>.<br>
Columns that are of special interest are colored <span style='color:green'>green</span>.<br>
 <span style='color:orange'>orange</span> columns will get a special attention.<br>
   

<ul>
    <li>name, essround, edition, proddate and respondents ID (columns <span style='color:red'>0:4</span>)</li>
    <li> column <span style='color:green'>5</span> = country of residence: keep!</li>
    <li> columns <span style='color:green'>6 till 11</span> revolve around <b>media and trust</b>: keep!</li>
    <li> columns 12 till 102 revolve around <b>politics</b>. Will have to delete columns here aswell, as I cant compare a French vote-code 2(front national) with a Dutch vote-code 2 (Labour party). These columns might come in handy when analyzing per country, but are in the way when analyzing them together. The columns to be deleted are: <span style='color:red'>25 till 50</span>, the same goes for <span style='color:red'>60 till 82</span> (which party do you feel closest to) </li>
    <li> columns <span style='color:green'>103 till 110</span> revolve around <b>subject wellbeing</b>: keep!</li>
    <li> columns <span style='color:green'>111 and 112</span> revolve around <b> social exclusion</b> (feeling attached to country/europe): keep!
    <li> columns 113 till 147 revolve around <b>religion</b>. <span style='color:red'>115 till 128</span> deal with current religion per country,<span style='color:red'> 131 till 144 </span>deal with past religion per country. Will delete those.</li>
    <li> columns <span style='color:green'>148 till 162</span> revolve around <b>discrimination</b>: keep!</li>
    <li> columns 163 till 174 revolve around <b>national and ethnic identity</b>, check how cntry mother/father (occur 2 times each) are encoded!</li>
    <li> columns <span style='color:green'>175 till 177</span> deal with <b>tolerance to refugeers</b>: keep!</li>
    <li> columns <span style='color:orange'>178 till 209</span> deal with <b>climate- and energy</b> related questions: predictor is in here!</li>
    <li> columns <span style='color:green'>210 till 248</span> are <b>wellfare/benefit</b> related questions: keep! </li>
    <li> columns 249, 250 and 251 are about leaving/entering/staying in the EU. check these columns out, might be difficult to deal with! </li>
    <li> 252 till 288 are about <b>household composition</b> (number of people, gender, ages and relation to participant).</li>
    <ul>
        <li> columns <span style='color:red'>254 till 264</span> deal with genders of people living regularly in the participants household, up to 12 persons. This means lots of columns with empty values. I could turn this into 'mixed/non-mixed'-variable, but this will be highly correlated with how many people make up the household (more people increasing the chance of mixed-household). Another approach is to turn it into a 'predominantly male / predominantly female / 50:50'-household but.. Not I do not suspect this to be of major influence on a participants view on climate change. I therefore decide to delete these columns </li>
        <li> 265 and 266 are resp age and year of birth of the respondent. I'll keep age and delete <span style='color:red'>266.</span>
        <li> columns <span style='color:red'>267 till 277</span> that deal with year of birth of people living regularly in the participants household, up to 12 persons. This means lots of columns with empty values. I will have to deal with this later. I could turn this into a column that would be indicative if a person has children (and therefore might be more worried about the planet). But that question is already asked, so I decide to delete these columns.</li>
        <li>columns <span style='color:red'>278 till 288</span> deal with the relationship of people living regularly in the participants household, up to 12 persons. This means lots of columns with empty values. I will also delete these also they will be higly correlated with having a partner or not (a question that's also asked).</li>
    </ul>
    <li> columns 289 till 493 concern <b>social demographics</b> like education </li>
    <ul>
        <li> columns <span style='color:red'>305 till 334</span> deal with the participants <b>own education</b>. Column 304 seems to be generalized for all countries, and is the column to keep.</li>
        <li> columns <span style='color:red'>379 till 408</span> deal in a similar way with the <b>partners education</b>. Column 378 seems to be generalized for all countries and is the column to keep.</li>
        <li> columns <span style='color:red'>430 till 460</span> and <span style='color:red'>462 till 490</span> deal with the <b>education of the participants father and mother </b>. All countries have their own encoding, which is too much to deal with right now. column 461 and 429 seem to be generalized for all country. So should be oke to keep those two and throw the other ones out.</li>
    </ul>
    <li> columns 494 and 495 concern first and second <b>ancestry</b>: keep</li>
    <li> columns 496 till 516 concern <b>human values</b> (important to be rich / creative / humble etc): keepers! </li>
    <li> place of interview: Eastm West Germany (column <span style='color:red'>519</span>) as it is only applicable to Germany and I will not dive into the regional differences there </li>
    <li>start of interview-columns(n=5), end of interview columns (n=5), interview length (columns <span style='color:red'>520:530</span>)</li> 
    <li>design <b>weights</b>, population size weight, analysis weight (columns <span style='color:orange'>531:534</span>)</li>
    <ul>
        <li> Note, I will store the weights in seperate variables before deleting them from the dataframe.</li>
    </ul>

#### Functions for cleaning

In [16]:
def check_NaNs(dataframe):
    """
    This function returns the percentage of NaNs per column
    """
    NaN_count = dataframe.isna().sum()
    NaN_perc = NaN_count/(len(dataframe))*100
    
    return NaN_perc.sort_values(ascending=False) 

In [17]:
def rows_to_remove(dataframe, lower, upper):
    """
    CONTEXT: To be able to apply MachineLearning algorithms, you first have to get rid of NaNs. Sometimes it's best to delete an entire column (when there are a lot of NaNs in this column).
    Sometimes it's better to get rid of some individual rows instead. 
    
    GOAL: This function identifies the row-number of NaN-elements that are present in columns with relatively little NaNs (you decide the lower&upper limit).

    IN: The dataframe from which rows should be dropped. 
    lower / upper: float (0-1) 

    OUT: It returns a list of the row numbers that you want to drop (note: these are the same as the indices if your indices run neatly from 0-end without missing numbers). 
    
    FOLLOW UP STEPS:
    If your dataframe has a neat index, or you can use dataframe.reset_index: please do and use the output of this function to delete the rows.
    If not, store the outcome of this function in a variable, and feed this to the function "indices_to_remove".
    """
    
    rows_to_del = []

    for i in range(len(dataframe.columns)):   
        if dataframe[dataframe.columns[i]].isna().sum() / len(dataframe) < upper and dataframe[dataframe.columns[i]].isna().sum() / len(dataframe) > lower: #check if between 0% & 5% is empty
            for j in range(len(dataframe)): #loop over the column, if you encounter an NaN: delete the row
                if pd.isna(dataframe.iloc[j,i])==True: #check if cell is empty (j = row, i = column)
                    rows_to_del.append(j) 
    
    return list(set(rows_to_del))

In [18]:
def indices_to_remove(dataframe, rows_to_remove_output):
    """
    NOTE: This is a workaround! Use if your computer can handle 'dataframe.reset_index' and you do need to drop rows based on the value of certain elements. 
    
    WHAT IT DOES:
    This function takes a dataframe from which rows should be dropped but the indices contain 'gaps' (eg 1,2,5,22,23, etc).
    Here's a workaround that lets you delete rows of which you don't know the index.

    IN: dataframe from which rows should be removed. The output from the "rows_to_remove"-function which is applied to the same dataframe as you use here.
    This output is basically a list of row_numbers that should be deleted. (row_numbers being: 0,1,2,3 without the gaps your indices have)
    
    OUT: list of indices you want to remove
       
    """
    
    len_df =len(dataframe)
    dataframe["row_ind"] = [*range(0,len_df)] #make a column with values running from 0,1,2,3 etc to the end of your dataframe
    
    rows_to_del = rows_to_remove_output
    
    rows_to_drop = list(set(rows_to_del)) #list with row numbers to drop (note: not indices, but first, fourth row etc)
    index_nrs = list(dataframe["row_ind"].index) #list with the index numbers. 

    index_to_drop = []

    for i in range(len(dataframe)):
        index_needed = index_nrs[i] #look up the index that belongs to the i'th row.
        if dataframe["row_ind"][index_needed] in rows_to_drop:
            index_to_drop.append(index_needed)
    
    return index_to_drop

#### NaN removal

First and foremost saveguard the weight-columns in their own variable. 

In [19]:
#assigning the complete weights-columns to their own parameters, keeping the original abbreviations. You never know if you won't need them!
dweight = climate["dweight"]
pspwght = climate["pspwght"]
pweight = climate["pweight"]
anweight = climate["anweight"]

In [20]:
# create a list with all column indices that have to go - leave the weights in.

cols_ind_drop = [*range(0,5,1), *range(25,51,1) ,*range(60,83,1) ,*range(115,129,1),*range(131,145,1), *range(254,265,1),*range(266,289,1),*range(305,335,1), *range(379,409,1), *range(430,461,1), *range(462,491,1), 519, *range(520,531,1)]


In [21]:
# first drop the columns from the overview, to be able to get a slightly shorter overview.
overview_minus = overview.drop(cols_ind_drop)
overview_minus

,shorthand,nom_or_scale,full_text
5,cntry,nominal,Country
6,nwspol,scale,"News about politics and current affairs, watching, reading or listening, in minutes"
7,netusoft,nominal,"Internet use, how often"
8,netustm,scale,"Internet use, how much time on typical day, in minutes"
9,ppltrst,nominal,Most people can be trusted or you can't be too careful
10,pplfair,nominal,"Most people try to take advantage of you, or try to be fair"
11,pplhlp,nominal,Most of the time people helpful or mostly looking out for themselves
12,polintr,nominal,How interested in politics
13,psppsgva,nominal,Political system allows people to have a say in what government does
14,actrolga,nominal,Able to take active role in political group


In [22]:
my_climate = climate.drop(climate.columns[cols_ind_drop], axis=1)

print(len(my_climate.columns))

pd.set_option('display.max_columns', None)
my_climate.head()

287


,cntry,nwspol,netusoft,netustm,ppltrst,pplfair,pplhlp,polintr,psppsgva,actrolga,psppipla,cptppola,trstprl,trstlgl,trstplc,trstplt,trstprt,trstep,trstun,vote,contplt,wrkprty,wrkorg,badge,sgnptit,pbldmn,bctprd,pstplonl,clsprty,prtdgcl,lrscale,stflife,stfeco,stfgov,stfdem,stfedu,stfhlth,gincdif,mnrgtjb,freehms,hmsfmlsh,hmsacld,euftf,imsmetn,imdfetn,impcntr,imbgeco,imueclt,imwbcnt,happy,sclmeet,inprdsc,sclact,crmvct,aesfdrk,health,hlthhmp,atchctr,atcherp,rlgblg,rlgdnm,rlgblge,rlgdnme,rlgdgr,rlgatnd,pray,dscrgrp,dscrrce,dscrntn,dscrrlg,dscrlng,dscretn,dscrage,dscrgnd,dscrsex,dscrdsb,dscroth,dscrdk,dscrref,dscrnap,dscrna,ctzcntr,ctzshipc,brncntr,cntbrthc,livecnta,lnghom1,lnghom2,blgetmg,facntr,fbrncntb,mocntr,mbrncntb,gvrfgap,rfgfrpc,rfgbfml,eneffap,rdcenr,cflsenr,elgcoal,elgngas,elghydr,elgnuc,elgsun,elgwind,elgbio,wrpwrct,wrenexp,wrdpimp,wrdpfos,wrntdis,wrinspw,wrtcfl,wrtratc,clmchng,clmthgt1,clmthgt2,ccnthum,ccrdprs,wrclmch,ccgdbd,lkredcc,lklmten,gvsrdcc,ownrdcc,inctxff,sbsrnen,banhhap,dfincac,smdfslv,uemplwk,slvpens,slvuemp,gvslvol,gvslvue,gvcldcr,sbstrec,sbprvpv,sbeqsoc,sbbsntx,sblazy,sblwcoa,imsclbn,uentrjb,lbenent,bennent,admub,ubpay,ubedu,ubunp,ub50pay,ub50edu,ub50unp,ub20pay,ub20edu,ub20unp,ubsppay,ubspedu,ubspunp,bnlwinc,eduunmp,wrkprbf,basinc,eusclbf,eudcnbf,lkuemp,lknemny,vteurmmb,vteumbgb,vteubcmb,hhmmb,gndr,agea,icpart1,rshpsts,rshpsfi,rshpsgb,lvgptnea,dvrcdeva,icpart2,iccohbt,marsts,marstfi,marstgb,maritalb,chldhm,chldhhe,domicil,edulvlb,eduyrs,pdwrk,edctn,uempla,uempli,dsbld,rtrd,cmsrv,hswrk,dngoth,dngref,dngdk,dngna,icomdng,mainact,mnactic,icpdwrk,crpdwk,pdjobev,pdjobyr,emplrel,emplno,wrkctra,estsz,jbspv,njbspv,wkdcorga,iorgact,wkhct,icwhct,wkhtot,nacer2,tporgwk,isco08,wrkac6m,uemp3m,uemp12m,uemp5yr,mbtru,hincsrca,hinctnta,hincfel,icpart3,edulvlpb,pdwrkp,edctnp,uemplap,uemplip,dsbldp,rtrdp,cmsrvp,hswrkp,dngothp,dngdkp,dngnapp,dngrefp,dngnap,icomdnp,mnactp,icppdwk,crpdwkp,isco08p,emprelp,wkhtotp,edulvlfb,edulvlmb,emprm14,occm14b,atncrse,anctry1,anctry2,ipcrtiv,imprich,ipeqopt,ipshabt,impsafe,impdiff,ipfrule,ipudrst,ipmodst,ipgdtim,impfree,iphlppl,ipsuces,ipstrgv,ipadvnt,ipbhprp,iprspot,iplylfr,impenv,imptrad,impfun,region,regunit,dweight,pspwght,pweight,anweight
0,AT,120.0,4.0,180.0,8.0,8.0,3.0,1.0,2.0,2.0,2.0,3.0,6.0,7.0,0.0,2.0,8.0,9.0,10.0,3.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,NaN,0.0,5.0,4.0,4.0,6.0,3.0,5.0,1.0,5.0,1.0,5.0,1.0,5.0,1.0,1.0,1.0,10.0,10.0,10.0,5.0,6.0,3.0,3.0,2.0,2.0,3.0,3.0,5.0,7.0,2.0,NaN,2.0,NaN,0.0,7.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,DE,2.0,DE,2010.0,GER,NaN,2.0,2.0,DE,2.0,DE,1.0,5.0,1.0,8.0,4.0,10.0,5.0,5.0,1.0,5.0,1.0,1.0,1.0,2.0,3.0,4.0,4.0,2.0,2.0,2.0,3.0,1.0,NaN,4.0,4.0,8.0,4.0,10.0,10.0,0.0,0.0,0.0,1.0,1.0,2.0,5.0,1.0,2.0,6.0,3.0,10.0,10.0,10.0,5.0,1.0,1.0,5.0,5.0,5.0,1.0,5.0,3.0,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,4.0,4.0,NaN,3.0,4.0,4.0,2.0,55.0,NaN,55.0,NaN,NaN,1.0,2.0,34.0,2.0,NaN,NaN,NaN,1.0,2.0,2.0,NaN,6.0,NaN,NaN,6.0,2.0,2.0,1.0,720.0,21.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,2.0,2.0,2.0,1.0,2015.0,1.0,NaN,2.0,1.0,2.0,NaN,5.0,0.0,25.0,1.0,40.0,72.0,2.0,2310.0,2.0,1.0,2.0,1.0,3.0,6.0,NaN,2.0,2.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,610.0,610.0,1.0,1.0,1.0,11070.0,NaN,2.0,6.0,1.0,2.0,2.0,3.0,5.0,1.0,2.0,4.0,3.0,1.0,2.0,6.0,3.0,6.0,1.0,1.0,1.0,6.0,3.0,AT13,2.0,0.611677,1.178496,0.370393,0.436506
1,AT,120.0,5.0,120.0,6.0,6.0,5.0,1.0,1.0,3.0,2.0,3.0,5.0,4.0,3.0,3.0,4.0,6.0,7.0,3.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,5.0,1.0,3.0,3.0,0.0,3.0,1.0,4.0,1.0,5.0,1.0,10.0,1.0,1.0,1.0,8.0,8.0,5.0,5.0,4.0,2.0,2.0,2.0,1.0,3.0,3.0,0.0,10.0,2.0,NaN,2.0,NaN,0.0,7.0,7.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,DE,2.0,DE,1994.0,GER,NaN,2.0,2.0,DE,2.0,DE,3.0,4.0,2.0,10.0,5.0,3.0,5.0,2.0,2.0,5.0,2.0,5.0,5.0,1.0,2.0,2.0,3.0,1.0,1.0,2.0,2.0,1.0,NaN,4.0,4.0,7.0,4.0,2.0,8.0,3.0,4.0,6.0,1.0,2.0,2.0,4.0,2.0,3.0,7.0,7.0,8.0,4.0,8.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,

In [23]:
#look for columns with lots of nans
check_NaNs(my_climate) #percentages!

clmthgt1    97.823687
rshpsgb     97.780882
marstfi     97.693018
marstgb     97.352829
rshpsfi     97.328047
ctzshipc    95.976299
vteumbgb    95.802825
mnactp      93.833780
cntbrthc    90.380066
livecnta    89.965530
rlgdnme     89.875414
emplno      89.481154
mainact     86.469011
mbrncntb    85.869737
lnghom2     85.358326
fbrncntb    85.265956
vteubcmb    82.456575
crpdwkp     77.542974
ub50unp     77.466375
ub50edu     77.024805
ubspunp     76.903147
ub50pay     76.894136
ub20unp     76.785996
ubspedu     76.562958
ubunp       76.508888
ubsppay     76.389483
ub20edu     76.324149
ub20pay     76.182216
ubedu       75.965936
ubpay       75.866808
anctry2     74.720076
njbspv      74.258229
uemp5yr     72.050375
uemp12m     72.045869
wkhtotp     67.253926
pdjobyr     66.156758
isco08p     64.489603
emprelp     63.719107
rlgblge     59.817064
prtdgcl     57.043729
pdjobev     56.744092
crpdwk      54.684930
marsts      49.739789
occm14b     42.758015
edulvlpb    42.523712
iccohbt   

Ouch, there are lot's of columns with lots of NaNs. For starters, I will ruthlessly delete all columns that have more than 40% NaNs.

In [24]:
# This: my_climate[my_climate.columns[2]]
# is the same as: # my_climate["netusoft"]

In [25]:
cols_to_del = []

for i in range(len(my_climate.columns)):   
    if my_climate[my_climate.columns[i]].isna().sum() / len(my_climate) >0.4: #check if >40% is empty
        cols_to_del.append(i)

In [26]:
#delete columns that are in cols_to_del from the my_climate dataframe
my_climate = my_climate.drop(my_climate.columns[cols_to_del], axis=1)

In [27]:
#check if it went right:
check_NaNs(my_climate)

chldhhe     35.147678
vteurmmb    33.385901
netustm     32.158064
eudcnbf     28.438507
eusclbf     26.584360
wrkctra     19.746773
wkhct       18.769009
hinctnta    17.892626
wkhtot      14.407372
lrscale     13.075901
rfgfrpc     11.706130
lvgptnea    11.320882
estsz       11.138396
lbenent     10.469282
edulvlfb    10.066010
nacer2      10.007435
isco08       9.597405
iorgact      9.214410
tporgwk      9.063465
wkdcorga     8.752563
basinc       8.549801
lknemny      8.504742
emplrel      8.398856
jbspv        8.385338
wrkac6m      8.252416
euftf        8.135265
icwhct       8.121747
wrkprbf      7.927997
trstep       7.801834
trstun       7.792822
sbbsntx      7.488679
gvsrdcc      7.454885
elgcoal      7.423345
lkredcc      7.333228
eduunmp      7.249870
elgbio       7.128213
ccgdbd       7.107937
uemplwk      6.889405
elgngas      6.824070
rfgbfml      6.727195
lklmten      6.706919
edulvlmb     6.641584
elgnuc       6.585261
bennent      6.537950
ownrdcc      6.157208
bnlwinc   

The extremes are gone. For the columns that have less than 5% missing values, I will delete rows that contain NaNs, and see what happens to the NaN-percentages of the other columns.

In [28]:
# NOTE: this takes quite long to compute. Enough time to make some tea.. drink it.. and make some more.. Be patient..  

rows_to_del = rows_to_remove(my_climate, 0, 0.05)        

In [29]:
print(len(rows_to_del)) #ouch, that's almost 25% of the participants. Go through with this, but also try to fill the gaps with 999 or 6666 or 7777 or something like that

19797


In [30]:
my_climate_short = my_climate.drop(rows_to_del, axis=0)

In [31]:
print(len(my_climate_short)) 

#check if that sums up right:
len(my_climate_short)+len(rows_to_del)==len(my_climate)

24590


True

In [32]:
#check out the percentages again
check_NaNs(my_climate_short)

chldhhe     36.177308
vteurmmb    29.479463
netustm     24.831232
eudcnbf     19.263928
eusclbf     18.255388
wrkctra     16.937780
wkhct       13.432290
lvgptnea    12.903619
hinctnta     9.926800
wkhtot       8.113054
nacer2       6.925580
isco08       6.742578
estsz        6.583977
edulvlfb     6.535177
tporgwk      5.965840
iorgact      5.933306
wkdcorga     5.815372
emplrel      5.734038
icwhct       5.693371
wrkac6m      5.668971
jbspv        5.636438
lknemny      5.441236
lrscale      5.124034
rfgfrpc      5.030500
lbenent      4.855632
edulvlmb     3.887759
trstep       2.818219
rfgbfml      2.753152
basinc       2.553884
wrkprbf      2.090281
trstun       2.017080
euftf        1.947946
eduunmp      1.764945
bennent      1.691745
uemplwk      1.598211
elgcoal      1.411143
bnlwinc      1.394876
elgngas      1.236275
sbbsntx      1.118341
elgnuc       0.971940
elgbio       0.967873
ccgdbd       0.544937
elghydr      0.431070
gvsrdcc      0.349736
lkredcc      0.309069
lklmten   

The deleting of rows affects the percentages of NaNs in other rows too. There are new columns that drop below the 5% NaNs. I will repeat the row-deleting-process.

In [33]:
#repeat the deleting process. It will - once again - take time (though a lot less than the run before). 
rows_to_del = rows_to_remove(my_climate_short, 0, 0.05)   

In [34]:
print(len(rows_to_del)) # that's almost another 6.000 participants gone. Here we go!

5951


My computer does not have enough power to reset the index of this dataframe (it's too large). I therefore can't use the rows_to_del directly in the drop() function. I have created a workaround. By feeding it into to the indices_to_remove function, I get the indices. Those I can drop.

In [35]:
# Note: this takes quite some time. This might be a good time to do some stretching.
index_to_drop = indices_to_remove(my_climate_short, rows_to_del)

In [36]:
len(index_to_drop) #check if this gets the same result as above.

5951

In [37]:
my_climate_shorter = my_climate_short.drop(index_to_drop, axis=0) #drop them.

In [38]:
check_NaNs(my_climate_shorter)

chldhhe     36.343151
vteurmmb    28.488653
netustm     22.275873
eudcnbf     17.066366
wrkctra     16.782016
eusclbf     16.095284
lvgptnea    13.562959
wkhct       12.645528
hinctnta     8.256881
wkhtot       7.382370
nacer2       6.717099
isco08       6.357637
estsz        6.159129
tporgwk      5.740651
iorgact      5.713826
wkdcorga     5.622619
emplrel      5.590429
icwhct       5.547508
wrkac6m      5.515317
jbspv        5.483127
lknemny      5.418746
edulvlfb     4.007726
lrscale      3.760931
rfgfrpc      1.636354
elghydr      0.000000
wrinspw      0.000000
dscretn      0.000000
wrpwrct      0.000000
dscrlng      0.000000
wrenexp      0.000000
wrdpimp      0.000000
wrdpfos      0.000000
wrntdis      0.000000
wrtratc      0.000000
wrtcfl       0.000000
dscrage      0.000000
clmchng      0.000000
clmthgt2     0.000000
dscrrlg      0.000000
dscrntn      0.000000
dscrrce      0.000000
dscrgrp      0.000000
pray         0.000000
rlgatnd      0.000000
rlgdgr       0.000000
elgbio    

I've now in total dropped around 26.000 rows, which is 42% of the data... 
The columns with the lowest percentage missing values (rfgfrpc) is now at 1,63% missing. It started out 11.7% missing. 
I decide to leave it at this, and drop the remaining columns that by now still contain NaNs.

In [39]:
perc_NaNs = check_NaNs(my_climate_shorter)
type(perc_NaNs)

pandas.core.series.Series

In [40]:
perc_NaNs_ind = perc_NaNs.reset_index()
perc_NaNs_ind.columns=['name','perc']
perc_NaNs_ind.head()        

,name,perc
0,chldhhe,36.343151
1,vteurmmb,28.488653
2,netustm,22.275873
3,eudcnbf,17.066366
4,wrkctra,16.782016


In [41]:
del_vars = perc_NaNs_ind.loc[perc_NaNs_ind['perc'] > 0, ['name']] #get names where % > 0 (these are the column-names you want to drop)
del_vars_list = del_vars["name"].tolist() #put those in a list
del_vars_list

['chldhhe',
 'vteurmmb',
 'netustm',
 'eudcnbf',
 'wrkctra',
 'eusclbf',
 'lvgptnea',
 'wkhct',
 'hinctnta',
 'wkhtot',
 'nacer2',
 'isco08',
 'estsz',
 'tporgwk',
 'iorgact',
 'wkdcorga',
 'emplrel',
 'icwhct',
 'wrkac6m',
 'jbspv',
 'lknemny',
 'edulvlfb',
 'lrscale',
 'rfgfrpc']

In [42]:
#drop the columns from the dataframe:
my_climate_shortest = my_climate_shorter.drop(del_vars_list, axis=1)
my_climate_shortest.head() #good to see clmchng is still in there ;) - the target variable.

,cntry,nwspol,netusoft,ppltrst,pplfair,pplhlp,polintr,psppsgva,actrolga,psppipla,cptppola,trstprl,trstlgl,trstplc,trstplt,trstprt,trstep,trstun,vote,contplt,wrkprty,wrkorg,badge,sgnptit,pbldmn,bctprd,pstplonl,clsprty,stflife,stfeco,stfgov,stfdem,stfedu,stfhlth,gincdif,mnrgtjb,freehms,hmsfmlsh,hmsacld,euftf,imsmetn,imdfetn,impcntr,imbgeco,imueclt,imwbcnt,happy,sclmeet,inprdsc,sclact,crmvct,aesfdrk,health,hlthhmp,atchctr,atcherp,rlgblg,rlgdgr,rlgatnd,pray,dscrgrp,dscrrce,dscrntn,dscrrlg,dscrlng,dscretn,dscrage,dscrgnd,dscrsex,dscrdsb,dscroth,dscrdk,dscrref,dscrnap,dscrna,ctzcntr,brncntr,lnghom1,blgetmg,facntr,mocntr,gvrfgap,rfgbfml,eneffap,rdcenr,cflsenr,elgcoal,elgngas,elghydr,elgnuc,elgsun,elgwind,elgbio,wrpwrct,wrenexp,wrdpimp,wrdpfos,wrntdis,wrinspw,wrtcfl,wrtratc,clmchng,clmthgt2,ccnthum,ccrdprs,wrclmch,ccgdbd,lkredcc,lklmten,gvsrdcc,ownrdcc,inctxff,sbsrnen,banhhap,dfincac,smdfslv,uemplwk,slvpens,slvuemp,gvslvol,gvslvue,gvcldcr,sbstrec,sbprvpv,sbeqsoc,sbbsntx,sblazy,sblwcoa,imsclbn,uentrjb,lbenent,bennent,admub,bnlwinc,eduunmp,wrkprbf,basinc,lkuemp,hhmmb,gndr,agea,icpart1,dvrcdeva,icpart2,maritalb,chldhm,domicil,edulvlb,eduyrs,pdwrk,edctn,uempla,uempli,dsbld,rtrd,cmsrv,hswrk,dngoth,dngref,dngdk,dngna,icomdng,mnactic,icpdwrk,uemp3m,mbtru,hincsrca,hincfel,icpart3,pdwrkp,edctnp,uemplap,uemplip,dsbldp,rtrdp,cmsrvp,hswrkp,dngothp,dngdkp,dngnapp,dngrefp,dngnap,edulvlmb,emprm14,atncrse,anctry1,ipcrtiv,imprich,ipeqopt,ipshabt,impsafe,impdiff,ipfrule,ipudrst,ipmodst,ipgdtim,impfree,iphlppl,ipsuces,ipstrgv,ipadvnt,ipbhprp,iprspot,iplylfr,impenv,imptrad,impfun,region,regunit,dweight,pspwght,pweight,anweight,row_ind
1,AT,120.0,5.0,6.0,6.0,5.0,1.0,1.0,3.0,2.0,3.0,5.0,4.0,3.0,3.0,4.0,6.0,7.0,3.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,5.0,1.0,3.0,3.0,0.0,3.0,1.0,4.0,1.0,5.0,1.0,10.0,1.0,1.0,1.0,8.0,8.0,5.0,5.0,4.0,2.0,2.0,2.0,1.0,3.0,3.0,0.0,10.0,2.0,0.0,7.0,7.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,GER,2.0,2.0,2.0,3.0,2.0,10.0,5.0,3.0,5.0,2.0,2.0,5.0,2.0,5.0,5.0,1.0,2.0,2.0,3.0,1.0,1.0,2.0,2.0,1.0,4.0,4.0,7.0,4.0,2.0,8.0,3.0,4.0,6.0,1.0,2.0,2.0,4.0,2.0,3.0,7.0,7.0,8.0,4.0,8.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,4.0,3.0,2.0,3.0,2.0,3.0,2.0,2.0,1.0,52.0,1.0,2.0,1.0,1.0,2.0,1.0,313.0,16.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,113.0,3.0,2.0,11070.0,3.0,6.0,2.0,3.0,5.0,3.0,5.0,3.0,4.0,3.0,2.0,3.0,3.0,4.0,5.0,5.0,2.0,2.0,2.0,5.0,5.0,AT13,2.0,1.223354,0.899472,0.370393,0.333158,1
2,AT,30.0,2.0,5.0,6.0,4.0,3.0,2.0,2.0,2.0,2.0,3.0,5.0,9.0,3.0,3.0,4.0,5.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,9.0,6.0,3.0,6.0,8.0,6.0,4.0,5.0,1.0,5.0,3.0,0.0,2.0,3.0,3.0,5.0,5.0,4.0,8.0,4.0,2.0,3.0,2.0,3.0,3.0,3.0,8.0,9.0,1.0,6.0,6.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,GER,2.0,1.0,1.0,3.0,3.0,9.0,4.0,3.0,5.0,4.0,2.0,5.0,1.0,1.0,4.0,3.0,4.0,3.0,3.0,3.0,2.0,3.0,3.0,1.0,4.0,4.0,8.0,4.0,2.0,6.0,1.0,3.0,1.0,4.0,2.0,2.0,2.0,3.0,3.0,7.0,8.0,10.0,8.0,8.0,4.0,2.0,5.0,4.0,2.0,3.0,3.0,3.0,5.0,2.0,4.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,68.0,2.0,2.0,2.0,6.0,2.0,4.0,322.0,13.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,2.0,1.0,2.0,4.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,113.0,1.0,2.0,11010.0,3.0,5.0,2.0,4.0,1.0,6.0,5.0,2.0,3.0,2.0,2.0,3.0,6.0,1.0,6.0,3.0,3.0,1.0,2.0,2.0,3.0,AT31,2.0,0.389058,0.315753,0.370393,0.116953,2
4,AT,30.0,5.0,5.0,5.0,7.0,3.0,3.0,3.0,4.0,1.0,7.0,7.0,10.0,6.0,7.0,9.0,10.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,10.0,7.0,5.0,9.0,10.0,10.0,1.0,5.0,1.0,5.0,1.0,8.0,2.0,2.0,1.0,2.0,5.0,5.0,5.0,6.0,4.0,3.0,2.0,2.0,1.0,3.0,8.0,8.0,1.0,10.0,5.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,TUR,1.0,2.0,2.0,2.0,2.0,10.0,6.0,10.0,5.0,2.0,3.0,3.0,1.0,1.0,3.0,1.0,1.0,1.0,3.0,1.0,1.0,4.0,5.0,1.0,2.0,4.0,6.0,4.0,5.0,10.0,10.0,2.0,7.0,3.0,1.0,4.0,4.0,2.0,5.0,8.0,3.0,10.0,10.0,10.0,3.0,1.0,1.0,2.0,2.0,5.0,2.0,5.0,2.0,4.0,3.0,3.0,2.0,1.0,3.0,4.0,5.0,2.0,20.0,2.0,2.

My computer can still not handle resetting the index. I will update the row_ind and store the dataframe.

In [43]:
my_climate_shortest["row_ind"]=[*range(0,(len(my_climate_shortest)),1)]

In [44]:
print(len(my_climate_shortest))
my_climate_shortest.tail() #check (start with 0, so should be 1 shorter than length)

18639


,cntry,nwspol,netusoft,ppltrst,pplfair,pplhlp,polintr,psppsgva,actrolga,psppipla,cptppola,trstprl,trstlgl,trstplc,trstplt,trstprt,trstep,trstun,vote,contplt,wrkprty,wrkorg,badge,sgnptit,pbldmn,bctprd,pstplonl,clsprty,stflife,stfeco,stfgov,stfdem,stfedu,stfhlth,gincdif,mnrgtjb,freehms,hmsfmlsh,hmsacld,euftf,imsmetn,imdfetn,impcntr,imbgeco,imueclt,imwbcnt,happy,sclmeet,inprdsc,sclact,crmvct,aesfdrk,health,hlthhmp,atchctr,atcherp,rlgblg,rlgdgr,rlgatnd,pray,dscrgrp,dscrrce,dscrntn,dscrrlg,dscrlng,dscretn,dscrage,dscrgnd,dscrsex,dscrdsb,dscroth,dscrdk,dscrref,dscrnap,dscrna,ctzcntr,brncntr,lnghom1,blgetmg,facntr,mocntr,gvrfgap,rfgbfml,eneffap,rdcenr,cflsenr,elgcoal,elgngas,elghydr,elgnuc,elgsun,elgwind,elgbio,wrpwrct,wrenexp,wrdpimp,wrdpfos,wrntdis,wrinspw,wrtcfl,wrtratc,clmchng,clmthgt2,ccnthum,ccrdprs,wrclmch,ccgdbd,lkredcc,lklmten,gvsrdcc,ownrdcc,inctxff,sbsrnen,banhhap,dfincac,smdfslv,uemplwk,slvpens,slvuemp,gvslvol,gvslvue,gvcldcr,sbstrec,sbprvpv,sbeqsoc,sbbsntx,sblazy,sblwcoa,imsclbn,uentrjb,lbenent,bennent,admub,bnlwinc,eduunmp,wrkprbf,basinc,lkuemp,hhmmb,gndr,agea,icpart1,dvrcdeva,icpart2,maritalb,chldhm,domicil,edulvlb,eduyrs,pdwrk,edctn,uempla,uempli,dsbld,rtrd,cmsrv,hswrk,dngoth,dngref,dngdk,dngna,icomdng,mnactic,icpdwrk,uemp3m,mbtru,hincsrca,hincfel,icpart3,pdwrkp,edctnp,uemplap,uemplip,dsbldp,rtrdp,cmsrvp,hswrkp,dngothp,dngdkp,dngnapp,dngrefp,dngnap,edulvlmb,emprm14,atncrse,anctry1,ipcrtiv,imprich,ipeqopt,ipshabt,impsafe,impdiff,ipfrule,ipudrst,ipmodst,ipgdtim,impfree,iphlppl,ipsuces,ipstrgv,ipadvnt,ipbhprp,iprspot,iplylfr,impenv,imptrad,impfun,region,regunit,dweight,pspwght,pweight,anweight,row_ind
44378,SI,120.0,1.0,5.0,3.0,8.0,2.0,1.0,1.0,2.0,1.0,5.0,5.0,8.0,3.0,1.0,1.0,6.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,10.0,5.0,0.0,0.0,6.0,2.0,1.0,4.0,2.0,3.0,4.0,8.0,3.0,3.0,3.0,5.0,2.0,2.0,8.0,3.0,4.0,3.0,2.0,2.0,3.0,2.0,9.0,5.0,2.0,1.0,7.0,7.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,SLV,2.0,1.0,1.0,4.0,4.0,10.0,5.0,7.0,5.0,3.0,1.0,5.0,1.0,1.0,2.0,4.0,4.0,4.0,4.0,4.0,3.0,3.0,5.0,1.0,4.0,4.0,6.0,4.0,2.0,7.0,5.0,5.0,2.0,3.0,1.0,1.0,5.0,2.0,3.0,2.0,2.0,8.0,8.0,8.0,4.0,2.0,2.0,4.0,2.0,2.0,5.0,2.0,2.0,2.0,2.0,4.0,3.0,3.0,1.0,55.0,2.0,1.0,66.0,1.0,2.0,1.0,1.0,2.0,4.0,321.0,11.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,2.0,2.0,2.0,4.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,213.0,3.0,2.0,14120.0,2.0,5.0,1.0,2.0,1.0,2.0,5.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,SI016,3.0,1.0,0.932756,0.134491,0.125447,18634
44382,SI,90.0,2.0,1.0,2.0,1.0,2.0,1.0,4.0,1.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,2.0,2.0,2.0,5.0,3.0,3.0,4.0,0.0,1.0,0.0,5.0,4.0,6.0,2.0,2.0,2.0,3.0,3.0,1.0,0.0,1.0,1.0,6.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,SLV,2.0,1.0,1.0,5.0,5.0,10.0,5.0,10.0,5.0,4.0,5.0,4.0,1.0,1.0,2.0,3.0,5.0,3.0,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,5.0,3.0,5.0,5.0,5.0,3.0,1.0,3.0,1.0,1.0,5.0,2.0,5.0,0.0,3.0,10.0,5.0,10.0,2.0,4.0,4.0,5.0,2.0,2.0,5.0,1.0,2.0,1.0,4.0,2.0,2.0,2.0,3.0,55.0,2.0,2.0,65.0,1.0,2.0,1.0,1.0,2.0,4.0,323.0,12.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,2.0,2.0,2.0,4.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,213.0,3.0,2.0,14120.0,1.0,2.0,3.0,2.0,1.0,1.0,5.0,4.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,SI016,3.0,1.0,0.835442,0.134491,0.112359,18635
44384,SI,240.0,5.0,3.0,3.0,4.0,2.0,2.0,2.0,1.0,2.0,3.0,5.0,9.0,1.0,1.0,2.0,5.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,5.0,3.0,0.0,3.0,6.0,3.0,1.0,5.0,2.0,5.0,2.0,8.0,2.0,2.0,2.0,8.0,10.0,7.0,7.0,4.0,4.0,2.0,2.0,1.0,4.0,1.0,9.0,9.0,2.0,0.0,7.0,7.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,SLV,2.0,1.0,1.0,3.0,2.0,10.0,6.0,10.0,4.0,1.0,1.0,4.0,2.0,2.0,4.0,1.0,4.0,4.0,5.0,2.0,2.0,2.0,3.0,1.0,4.0,3.0,7.0,4.0,1.0,8.0,6.0,2.0,4.0,1.0,1.0,2.0,3.0,1.0,4.0,2.0,2.0,10.0,8.0,10.0,2.0,1.0,1.0,2.0,3.0,4.0,2.0,3.0,4.0,2.0,3.0,1.0,2.0,4.0,4.0,1.0,3.0,1.0,63

In [45]:
# STORE IN CSV
my_climate_shortest.to_csv('./data/climate_shortened.csv')

#### NaN filling
Filling NaNs is another way to get rid of them. 

Sounds easy enough right? Have to make sure that: my computer can handle it (move slowly), and fill it with numbers not yet used. Basically all numbers from 0 to 9999 are taken. eg:

4444: schooling system in USSR<br>
55/5555: other<br>
66/6666: not applicable<br>
7/77/7777: refusal<br>
8/88/8888: Don’t know<br>
9/99/9999:No answer<br>
0,10,100,110,200,210,300,310, 1000-9629: ISCO08<br>

To be on the safe side, I will fill the NaNs that is not too much 'out of range' of the most common answers (0-10) and has a pattern that is easy to remember: <b>0.12345</b>


In [46]:
climate.tail()

,name,essround,edition,proddate,idno,cntry,nwspol,netusoft,netustm,ppltrst,pplfair,pplhlp,polintr,psppsgva,actrolga,psppipla,cptppola,trstprl,trstlgl,trstplc,trstplt,trstprt,trstep,trstun,vote,prtvtbat,prtvtcbe,prtvtfch,prtvtdcz,prtvede1,prtvede2,prtvtfee,prtvtdes,prtvtdfi,prtvtcfr,prtvtbgb,prtvtehu,prtvtbie,prtvtcil,prtvtbis,prtvtbit,prtvblt1,prtvblt2,prtvblt3,prtvtfnl,prtvtbno,prtvtdpl,prtvtcpt,prtvtdru,prtvtbse,prtvtesi,contplt,wrkprty,wrkorg,badge,sgnptit,pbldmn,bctprd,pstplonl,clsprty,prtclcat,prtclcbe,prtclfch,prtcldcz,prtclede,prtclfee,prtclees,prtcldfi,prtclefr,prtclbgb,prtclfhu,prtcldie,prtcldil,prtclbis,prtclcit,prtclblt,prtclenl,prtclbno,prtclgpl,prtclept,prtcldru,prtclbse,prtclesi,prtdgcl,lrscale,stflife,stfeco,stfgov,stfdem,stfedu,stfhlth,gincdif,mnrgtjb,freehms,hmsfmlsh,hmsacld,euftf,imsmetn,imdfetn,impcntr,imbgeco,imueclt,imwbcnt,happy,sclmeet,inprdsc,sclact,crmvct,aesfdrk,health,hlthhmp,atchctr,atcherp,rlgblg,rlgdnm,rlgdnbat,rlgdnbe,rlgdnach,rlgdnade,rlgdnafi,rlgdngb,rlgdnhu,rlgdnie,rlgdnis,rlgdnlt,rlgdnnl,rlgdnno,rlgdnapl,rlgdnase,rlgblge,rlgdnme,rlgdebat,rlgdebe,rlgdeach,rlgdeade,rlgdeafi,rlgdegb,rlgdehu,rlgdeie,rlgdeis,rlgdelt,rlgdenl,rlgdeno,rlgdeapl,rlgdease,rlgdgr,rlgatnd,pray,dscrgrp,dscrrce,dscrntn,dscrrlg,dscrlng,dscretn,dscrage,dscrgnd,dscrsex,dscrdsb,dscroth,dscrdk,dscrref,dscrnap,dscrna,ctzcntr,ctzshipc,brncntr,cntbrthc,livecnta,lnghom1,lnghom2,blgetmg,facntr,fbrncntb,mocntr,mbrncntb,gvrfgap,rfgfrpc,rfgbfml,eneffap,rdcenr,cflsenr,elgcoal,elgngas,elghydr,elgnuc,elgsun,elgwind,elgbio,wrpwrct,wrenexp,wrdpimp,wrdpfos,wrntdis,wrinspw,wrtcfl,wrtratc,clmchng,clmthgt1,clmthgt2,ccnthum,ccrdprs,wrclmch,ccgdbd,lkredcc,lklmten,gvsrdcc,ownrdcc,inctxff,sbsrnen,banhhap,dfincac,smdfslv,uemplwk,slvpens,slvuemp,gvslvol,gvslvue,gvcldcr,sbstrec,sbprvpv,sbeqsoc,sbbsntx,sblazy,sblwcoa,imsclbn,uentrjb,lbenent,bennent,admub,ubpay,ubedu,ubunp,ub50pay,ub50edu,ub50unp,ub20pay,ub20edu,ub20unp,ubsppay,ubspedu,ubspunp,bnlwinc,eduunmp,wrkprbf,basinc,eusclbf,eudcnbf,lkuemp,lknemny,vteurmmb,vteumbgb,vteubcmb,hhmmb,gndr,gndr2,gndr3,gndr4,gndr5,gndr6,gndr7,gndr8,gndr9,gndr10,gndr11,gndr12,agea,yrbrn,yrbrn2,yrbrn3,yrbrn4,yrbrn5,yrbrn6,yrbrn7,yrbrn8,yrbrn9,yrbrn10,yrbrn11,yrbrn12,rshipa2,rshipa3,rshipa4,rshipa5,rshipa6,rshipa7,rshipa8,rshipa9,rshipa10,rshipa11,rshipa12,icpart1,rshpsts,rshpsfi,rshpsgb,lvgptnea,dvrcdeva,icpart2,iccohbt,marsts,marstfi,marstgb,maritalb,chldhm,chldhhe,domicil,edulvlb,eisced,edlveat,edlvebe,edlvdch,edlvdcz,edubde1,eduade2,eduade3,edlvdee,edlvges,edlvdfi,edlvdfr,edubgb1,eduagb2,edagegb,edlvdhu,edlvdie,edubil1,eduail2,edlvdis,edlvdit,edlvdlt,edlvenl,edlvdno,edlvepl,edupl2,edlvdpt,edlvdru,edlvdse,edlvesi,eduyrs,pdwrk,edctn,uempla,uempli,dsbld,rtrd,cmsrv,hswrk,dngoth,dngref,dngdk,dngna,icomdng,mainact,mnactic,icpdwrk,crpdwk,pdjobev,pdjobyr,emplrel,emplno,wrkctra,estsz,jbspv,njbspv,wkdcorga,iorgact,wkhct,icwhct,wkhtot,nacer2,tporgwk,isco08,wrkac6m,uemp3m,uemp12m,uemp5yr,mbtru,hincsrca,hinctnta,hincfel,icpart3,edulvlpb,eiscedp,edlvpeat,edlvpebe,edlvpdch,edlvpdcz,edupbde1,edupade2,edupade3,edlvpdee,edlvpfes,edlvpdfi,edlvpdfr,edupbgb1,edupagb2,edagepgb,edlvpdhu,edlvpdie,edupbil1,edupail2,edlvpdis,edlvpdit,edlvpdlt,edlvpenl,edlvpdno,edlvpepl,eduppl2,edlvpdpt,edlvpdru,edlvpdse,edlvpesi,pdwrkp,edctnp,uemplap,uemplip,dsbldp,rtrdp,cmsrvp,hswrkp,dngothp,dngdkp,dngnapp,dngrefp,dngnap,icomdnp,mnactp,icppdwk,crpdwkp,isco08p,emprelp,wkhtotp,edulvlfb,eiscedf,edlvfeat,edlvfebe,edlvfdch,edlvfdcz,edufbde1,edufade2,edufade3,edlvfdee,edlvffes,edlvfdfi,edlvfdfr,edufbgb1,edufagb2,edagefgb,edlvfdhu,edlvfdie,edufbil1,edufail2,edlvfdis,edlvfdit,edlvfdlt,edlvfenl,edlvfdno,edlvfepl,edlvfdpt,edlvfdru,edlvfdse,edlvfesi,emprf14,occf14b,edulvlmb,eiscedm,edlvmeat,edlvmebe,edlvmdch,edlvmdcz,edumbde1,edumade2,edumade3,edlvmdee,edlvmfes,edlvmdfi,edlvmdfr,edumbgb1,edumagb2,edagemgb,edlvmdhu,edlvmdie,edumbil1,edumail2,edlvmdis,edlvmdit,edlvmdlt,edlvmenl,edlvmdno,edlvmepl,edlvmdpt,edlvmdru,edlvmdse,edlvmesi,emprm14,occm14b,atncrse,anctry1,anctry2,ip

In [47]:
check_NaNs(climate) #check NaNs. 

rshipa12    99.988735
yrbrn12     99.988735
gndr12      99.988735
yrbrn11     99.979724
gndr11      99.977471
rshipa11    99.977471
rlgdelt     99.941424
yrbrn10     99.936918
gndr10      99.925654
rshipa10    99.923401
rlgdehu     99.882849
yrbrn9      99.867078
gndr9       99.846802
rshipa9     99.844549
edupail2    99.806250
rlgdeapl    99.788226
eduail2     99.682339
yrbrn8      99.653052
rlgdease    99.637281
gndr8       99.628270
rshipa8     99.619258
rlgdeis     99.567441
edufail2    99.515624
rlgdeafi    99.504359
rlgdebat    99.495348
edumail2    99.484083
rlgdeach    99.407484
rlgdeno     99.382702
rlgdegb     99.337644
rlgdeie     99.258792
rlgdebe     99.215987
rlgdnis     99.069547
rlgdeade    99.058283
prtclesi    99.035754
rlgdenl     99.006466
prtclbis    99.004213
yrbrn7      98.997454
gndr7       98.950143
rshipa7     98.934373
prtclgpl    98.909591
prtclblt    98.875797
rlgdnnl     98.754140
rlgdnase    98.736116
edlvpdis    98.691058
prtclfhu    98.641494
prtvtesi  

Quite some columns are >90% empty. I will get rid of those anyway, to save some computing power when filling the gaps.

In [48]:
cols_to_del = []

for i in range(len(climate.columns)):   
    if climate[climate.columns[i]].isna().sum() / len(climate) > 0.9: #check if >90% is empty
        cols_to_del.append(i)

In [49]:
climate_cut = climate.drop(climate.columns[cols_to_del], axis=1)

In [50]:
print(len(climate.columns), "--->" , len(climate_cut.columns))

535 ---> 310


Some other columns will also have to go regardless of NaNs, as they have no meaning. These are columns 0 till 4. (note: will keep the weights!)

In [51]:
climate_cut2 = climate_cut.drop(climate_cut.columns[[0,1,2,3,4]], axis=1)
climate_cut2.head()

,cntry,nwspol,netusoft,netustm,ppltrst,pplfair,pplhlp,polintr,psppsgva,actrolga,psppipla,cptppola,trstprl,trstlgl,trstplc,trstplt,trstprt,trstep,trstun,vote,contplt,wrkprty,wrkorg,badge,sgnptit,pbldmn,bctprd,pstplonl,clsprty,prtdgcl,lrscale,stflife,stfeco,stfgov,stfdem,stfedu,stfhlth,gincdif,mnrgtjb,freehms,hmsfmlsh,hmsacld,euftf,imsmetn,imdfetn,impcntr,imbgeco,imueclt,imwbcnt,happy,sclmeet,inprdsc,sclact,crmvct,aesfdrk,health,hlthhmp,atchctr,atcherp,rlgblg,rlgdnm,rlgblge,rlgdnme,rlgdgr,rlgatnd,pray,dscrgrp,dscrrce,dscrntn,dscrrlg,dscrlng,dscretn,dscrage,dscrgnd,dscrsex,dscrdsb,dscroth,dscrdk,dscrref,dscrnap,dscrna,ctzcntr,brncntr,livecnta,lnghom1,lnghom2,blgetmg,facntr,fbrncntb,mocntr,mbrncntb,gvrfgap,rfgfrpc,rfgbfml,eneffap,rdcenr,cflsenr,elgcoal,elgngas,elghydr,elgnuc,elgsun,elgwind,elgbio,wrpwrct,wrenexp,wrdpimp,wrdpfos,wrntdis,wrinspw,wrtcfl,wrtratc,clmchng,clmthgt2,ccnthum,ccrdprs,wrclmch,ccgdbd,lkredcc,lklmten,gvsrdcc,ownrdcc,inctxff,sbsrnen,banhhap,dfincac,smdfslv,uemplwk,slvpens,slvuemp,gvslvol,gvslvue,gvcldcr,sbstrec,sbprvpv,sbeqsoc,sbbsntx,sblazy,sblwcoa,imsclbn,uentrjb,lbenent,bennent,admub,ubpay,ubedu,ubunp,ub50pay,ub50edu,ub50unp,ub20pay,ub20edu,ub20unp,ubsppay,ubspedu,ubspunp,bnlwinc,eduunmp,wrkprbf,basinc,eusclbf,eudcnbf,lkuemp,lknemny,vteurmmb,vteubcmb,hhmmb,gndr,gndr2,gndr3,gndr4,agea,yrbrn,yrbrn2,yrbrn3,yrbrn4,rshipa2,rshipa3,rshipa4,icpart1,rshpsts,lvgptnea,dvrcdeva,icpart2,iccohbt,marsts,maritalb,chldhm,chldhhe,domicil,edulvlb,eisced,eduyrs,pdwrk,edctn,uempla,uempli,dsbld,rtrd,cmsrv,hswrk,dngoth,dngref,dngdk,dngna,icomdng,mainact,mnactic,icpdwrk,crpdwk,pdjobev,pdjobyr,emplrel,emplno,wrkctra,estsz,jbspv,njbspv,wkdcorga,iorgact,wkhct,icwhct,wkhtot,nacer2,tporgwk,isco08,wrkac6m,uemp3m,uemp12m,uemp5yr,mbtru,hincsrca,hinctnta,hincfel,icpart3,edulvlpb,eiscedp,pdwrkp,edctnp,uemplap,uemplip,dsbldp,rtrdp,cmsrvp,hswrkp,dngothp,dngdkp,dngnapp,dngrefp,dngnap,icomdnp,icppdwk,crpdwkp,isco08p,emprelp,wkhtotp,edulvlfb,eiscedf,emprf14,occf14b,edulvlmb,eiscedm,emprm14,occm14b,atncrse,anctry1,anctry2,ipcrtiv,imprich,ipeqopt,ipshabt,impsafe,impdiff,ipfrule,ipudrst,ipmodst,ipgdtim,impfree,iphlppl,ipsuces,ipstrgv,ipadvnt,ipbhprp,iprspot,iplylfr,impenv,imptrad,impfun,region,regunit,inwdds,inwmms,inwyys,inwshh,inwsmm,inwdde,inwmme,inwyye,inwehh,inwemm,inwtm,dweight,pspwght,pweight,anweight
0,AT,120.0,4.0,180.0,8.0,8.0,3.0,1.0,2.0,2.0,2.0,3.0,6.0,7.0,0.0,2.0,8.0,9.0,10.0,3.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,NaN,0.0,5.0,4.0,4.0,6.0,3.0,5.0,1.0,5.0,1.0,5.0,1.0,5.0,1.0,1.0,1.0,10.0,10.0,10.0,5.0,6.0,3.0,3.0,2.0,2.0,3.0,3.0,5.0,7.0,2.0,NaN,2.0,NaN,0.0,7.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,2010.0,GER,NaN,2.0,2.0,DE,2.0,DE,1.0,5.0,1.0,8.0,4.0,10.0,5.0,5.0,1.0,5.0,1.0,1.0,1.0,2.0,3.0,4.0,4.0,2.0,2.0,2.0,3.0,1.0,4.0,4.0,8.0,4.0,10.0,10.0,0.0,0.0,0.0,1.0,1.0,2.0,5.0,1.0,2.0,6.0,3.0,10.0,10.0,10.0,5.0,1.0,1.0,5.0,5.0,5.0,1.0,5.0,3.0,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,4.0,4.0,NaN,3.0,4.0,4.0,2.0,55.0,NaN,55.0,NaN,1.0,2.0,NaN,NaN,NaN,34.0,1982.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0,2.0,2.0,NaN,6.0,6.0,2.0,2.0,1.0,720.0,7.0,21.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,2.0,2.0,2.0,1.0,2015.0,1.0,NaN,2.0,1.0,2.0,NaN,5.0,0.0,25.0,1.0,40.0,72.0,2.0,2310.0,2.0,1.0,2.0,1.0,3.0,6.0,NaN,2.0,2.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,610.0,6.0,1.0,3.0,610.0,6.0,1.0,1.0,1.0,11070.0,NaN,2.0,6.0,1.0,2.0,2.0,3.0,5.0,1.0,2.0,4.0,3.0,1.0,2.0,6.0,3.0,6.0,1.0,1.0,1.0,6.0,3.0,AT13,2.0,5.0,12.0,2016.0,7.0,37.0,5.0,12.0,2016.0,8.0,33.0,50.0,0.611677,1.178496,0.370393,0.436506
1,AT,120.0,5.0,120.0,6.0,6.0,5.0,1.0,1.0,3.0,2.0,3.0,5.0,4.0,3.0,3.0,4.0,6.0,7.0,3.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,5.0,1.0,3.0,3.0,0.0,3.0,1.0,4.0,1.0,5.0,1.0,10.0,1.0,1.0,1.0,8.0,8.0,5.0,5.0,4.0,2.0,2.0,2.0,1.0,3.0,3.0,0.0,10.0,2.0,NaN,2.0,NaN,0.0,7.0,7.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,1994.0,GER,NaN,2.0,2.0,DE,2.0,DE,3.0,4.

NaN-values in string-columns, should be filled with strings. I'll check if "ZZ" already exists in the columns, if not, I'll use ZZ.

In [118]:
#select stringcolumns
str_cols = climate_cut2.select_dtypes(include='object').columns
str_cols

Index(['cntry', 'lnghom1', 'lnghom2', 'fbrncntb', 'mbrncntb', 'region'], dtype='object')

In [129]:
# check if it's save to fill empty values with ZZ 
df_str_cols = climate_cut2.select_dtypes(include='object')

for i in str_cols:
    uniques = df_str_cols[i].unique() #get unique values per column
    if "ZZ" in uniques: #check if ZZ is in there
        print("ZZ here!")
    else: 
        print("no ZZ!")

no ZZ!
no ZZ!
no ZZ!
no ZZ!
no ZZ!
no ZZ!


"ZZ" does not occur, so I can fill those NaNs.

In [133]:
climate_cut2[str_cols] = climate_cut2[str_cols].fillna("ZZ")

In [135]:
climate_cut2.head(15) #check

,cntry,nwspol,netusoft,netustm,ppltrst,pplfair,pplhlp,polintr,psppsgva,actrolga,psppipla,cptppola,trstprl,trstlgl,trstplc,trstplt,trstprt,trstep,trstun,vote,contplt,wrkprty,wrkorg,badge,sgnptit,pbldmn,bctprd,pstplonl,clsprty,prtdgcl,lrscale,stflife,stfeco,stfgov,stfdem,stfedu,stfhlth,gincdif,mnrgtjb,freehms,hmsfmlsh,hmsacld,euftf,imsmetn,imdfetn,impcntr,imbgeco,imueclt,imwbcnt,happy,sclmeet,inprdsc,sclact,crmvct,aesfdrk,health,hlthhmp,atchctr,atcherp,rlgblg,rlgdnm,rlgblge,rlgdnme,rlgdgr,rlgatnd,pray,dscrgrp,dscrrce,dscrntn,dscrrlg,dscrlng,dscretn,dscrage,dscrgnd,dscrsex,dscrdsb,dscroth,dscrdk,dscrref,dscrnap,dscrna,ctzcntr,brncntr,livecnta,lnghom1,lnghom2,blgetmg,facntr,fbrncntb,mocntr,mbrncntb,gvrfgap,rfgfrpc,rfgbfml,eneffap,rdcenr,cflsenr,elgcoal,elgngas,elghydr,elgnuc,elgsun,elgwind,elgbio,wrpwrct,wrenexp,wrdpimp,wrdpfos,wrntdis,wrinspw,wrtcfl,wrtratc,clmchng,clmthgt2,ccnthum,ccrdprs,wrclmch,ccgdbd,lkredcc,lklmten,gvsrdcc,ownrdcc,inctxff,sbsrnen,banhhap,dfincac,smdfslv,uemplwk,slvpens,slvuemp,gvslvol,gvslvue,gvcldcr,sbstrec,sbprvpv,sbeqsoc,sbbsntx,sblazy,sblwcoa,imsclbn,uentrjb,lbenent,bennent,admub,ubpay,ubedu,ubunp,ub50pay,ub50edu,ub50unp,ub20pay,ub20edu,ub20unp,ubsppay,ubspedu,ubspunp,bnlwinc,eduunmp,wrkprbf,basinc,eusclbf,eudcnbf,lkuemp,lknemny,vteurmmb,vteubcmb,hhmmb,gndr,gndr2,gndr3,gndr4,agea,yrbrn,yrbrn2,yrbrn3,yrbrn4,rshipa2,rshipa3,rshipa4,icpart1,rshpsts,lvgptnea,dvrcdeva,icpart2,iccohbt,marsts,maritalb,chldhm,chldhhe,domicil,edulvlb,eisced,eduyrs,pdwrk,edctn,uempla,uempli,dsbld,rtrd,cmsrv,hswrk,dngoth,dngref,dngdk,dngna,icomdng,mainact,mnactic,icpdwrk,crpdwk,pdjobev,pdjobyr,emplrel,emplno,wrkctra,estsz,jbspv,njbspv,wkdcorga,iorgact,wkhct,icwhct,wkhtot,nacer2,tporgwk,isco08,wrkac6m,uemp3m,uemp12m,uemp5yr,mbtru,hincsrca,hinctnta,hincfel,icpart3,edulvlpb,eiscedp,pdwrkp,edctnp,uemplap,uemplip,dsbldp,rtrdp,cmsrvp,hswrkp,dngothp,dngdkp,dngnapp,dngrefp,dngnap,icomdnp,icppdwk,crpdwkp,isco08p,emprelp,wkhtotp,edulvlfb,eiscedf,emprf14,occf14b,edulvlmb,eiscedm,emprm14,occm14b,atncrse,anctry1,anctry2,ipcrtiv,imprich,ipeqopt,ipshabt,impsafe,impdiff,ipfrule,ipudrst,ipmodst,ipgdtim,impfree,iphlppl,ipsuces,ipstrgv,ipadvnt,ipbhprp,iprspot,iplylfr,impenv,imptrad,impfun,region,regunit,inwdds,inwmms,inwyys,inwshh,inwsmm,inwdde,inwmme,inwyye,inwehh,inwemm,inwtm,dweight,pspwght,pweight,anweight
0,AT,120.0,4.0,180.0,8.0,8.0,3.0,1.0,2.0,2.0,2.0,3.0,6.0,7.0,0.0,2.0,8.0,9.0,10.0,3.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,NaN,0.0,5.0,4.0,4.0,6.0,3.0,5.0,1.0,5.0,1.0,5.0,1.0,5.0,1.0,1.0,1.0,10.0,10.0,10.0,5.0,6.0,3.0,3.0,2.0,2.0,3.0,3.0,5.0,7.0,2.0,NaN,2.0,NaN,0.0,7.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,2010.0,GER,ZZ,2.0,2.0,DE,2.0,DE,1.0,5.0,1.0,8.0,4.0,10.0,5.0,5.0,1.0,5.0,1.0,1.0,1.0,2.0,3.0,4.0,4.0,2.0,2.0,2.0,3.0,1.0,4.0,4.0,8.0,4.0,10.0,10.0,0.0,0.0,0.0,1.0,1.0,2.0,5.0,1.0,2.0,6.0,3.0,10.0,10.0,10.0,5.0,1.0,1.0,5.0,5.0,5.0,1.0,5.0,3.0,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,4.0,4.0,NaN,3.0,4.0,4.0,2.0,55.0,NaN,55.0,NaN,1.0,2.0,NaN,NaN,NaN,34.0,1982.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0,2.0,2.0,NaN,6.0,6.0,2.0,2.0,1.0,720.0,7.0,21.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,2.0,2.0,2.0,1.0,2015.0,1.0,NaN,2.0,1.0,2.0,NaN,5.0,0.0,25.0,1.0,40.0,72.0,2.0,2310.0,2.0,1.0,2.0,1.0,3.0,6.0,NaN,2.0,2.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,610.0,6.0,1.0,3.0,610.0,6.0,1.0,1.0,1.0,11070.0,NaN,2.0,6.0,1.0,2.0,2.0,3.0,5.0,1.0,2.0,4.0,3.0,1.0,2.0,6.0,3.0,6.0,1.0,1.0,1.0,6.0,3.0,AT13,2.0,5.0,12.0,2016.0,7.0,37.0,5.0,12.0,2016.0,8.0,33.0,50.0,0.611677,1.178496,0.370393,0.436506
1,AT,120.0,5.0,120.0,6.0,6.0,5.0,1.0,1.0,3.0,2.0,3.0,5.0,4.0,3.0,3.0,4.0,6.0,7.0,3.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,5.0,1.0,3.0,3.0,0.0,3.0,1.0,4.0,1.0,5.0,1.0,10.0,1.0,1.0,1.0,8.0,8.0,5.0,5.0,4.0,2.0,2.0,2.0,1.0,3.0,3.0,0.0,10.0,2.0,NaN,2.0,NaN,0.0,7.0,7.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,1994.0,GER,ZZ,2.0,2.0,DE,2.0,DE,3.0,4.0,

Now fill the numerical columns. As most values lie somewhere between 0 and 10, but higher values do occur aswell, I will fill the NaNs with "0.12345": it's recognizable, easy to memorize and yet not too far off the scale. 

In [157]:
climate_fill = climate_cut2.fillna(0.12345) 

In [158]:
climate_fill.head()

,cntry,nwspol,netusoft,netustm,ppltrst,pplfair,pplhlp,polintr,psppsgva,actrolga,psppipla,cptppola,trstprl,trstlgl,trstplc,trstplt,trstprt,trstep,trstun,vote,contplt,wrkprty,wrkorg,badge,sgnptit,pbldmn,bctprd,pstplonl,clsprty,prtdgcl,lrscale,stflife,stfeco,stfgov,stfdem,stfedu,stfhlth,gincdif,mnrgtjb,freehms,hmsfmlsh,hmsacld,euftf,imsmetn,imdfetn,impcntr,imbgeco,imueclt,imwbcnt,happy,sclmeet,inprdsc,sclact,crmvct,aesfdrk,health,hlthhmp,atchctr,atcherp,rlgblg,rlgdnm,rlgblge,rlgdnme,rlgdgr,rlgatnd,pray,dscrgrp,dscrrce,dscrntn,dscrrlg,dscrlng,dscretn,dscrage,dscrgnd,dscrsex,dscrdsb,dscroth,dscrdk,dscrref,dscrnap,dscrna,ctzcntr,brncntr,livecnta,lnghom1,lnghom2,blgetmg,facntr,fbrncntb,mocntr,mbrncntb,gvrfgap,rfgfrpc,rfgbfml,eneffap,rdcenr,cflsenr,elgcoal,elgngas,elghydr,elgnuc,elgsun,elgwind,elgbio,wrpwrct,wrenexp,wrdpimp,wrdpfos,wrntdis,wrinspw,wrtcfl,wrtratc,clmchng,clmthgt2,ccnthum,ccrdprs,wrclmch,ccgdbd,lkredcc,lklmten,gvsrdcc,ownrdcc,inctxff,sbsrnen,banhhap,dfincac,smdfslv,uemplwk,slvpens,slvuemp,gvslvol,gvslvue,gvcldcr,sbstrec,sbprvpv,sbeqsoc,sbbsntx,sblazy,sblwcoa,imsclbn,uentrjb,lbenent,bennent,admub,ubpay,ubedu,ubunp,ub50pay,ub50edu,ub50unp,ub20pay,ub20edu,ub20unp,ubsppay,ubspedu,ubspunp,bnlwinc,eduunmp,wrkprbf,basinc,eusclbf,eudcnbf,lkuemp,lknemny,vteurmmb,vteubcmb,hhmmb,gndr,gndr2,gndr3,gndr4,agea,yrbrn,yrbrn2,yrbrn3,yrbrn4,rshipa2,rshipa3,rshipa4,icpart1,rshpsts,lvgptnea,dvrcdeva,icpart2,iccohbt,marsts,maritalb,chldhm,chldhhe,domicil,edulvlb,eisced,eduyrs,pdwrk,edctn,uempla,uempli,dsbld,rtrd,cmsrv,hswrk,dngoth,dngref,dngdk,dngna,icomdng,mainact,mnactic,icpdwrk,crpdwk,pdjobev,pdjobyr,emplrel,emplno,wrkctra,estsz,jbspv,njbspv,wkdcorga,iorgact,wkhct,icwhct,wkhtot,nacer2,tporgwk,isco08,wrkac6m,uemp3m,uemp12m,uemp5yr,mbtru,hincsrca,hinctnta,hincfel,icpart3,edulvlpb,eiscedp,pdwrkp,edctnp,uemplap,uemplip,dsbldp,rtrdp,cmsrvp,hswrkp,dngothp,dngdkp,dngnapp,dngrefp,dngnap,icomdnp,icppdwk,crpdwkp,isco08p,emprelp,wkhtotp,edulvlfb,eiscedf,emprf14,occf14b,edulvlmb,eiscedm,emprm14,occm14b,atncrse,anctry1,anctry2,ipcrtiv,imprich,ipeqopt,ipshabt,impsafe,impdiff,ipfrule,ipudrst,ipmodst,ipgdtim,impfree,iphlppl,ipsuces,ipstrgv,ipadvnt,ipbhprp,iprspot,iplylfr,impenv,imptrad,impfun,region,regunit,inwdds,inwmms,inwyys,inwshh,inwsmm,inwdde,inwmme,inwyye,inwehh,inwemm,inwtm,dweight,pspwght,pweight,anweight
0,AT,120.0,4.0,180.00000,8.0,8.0,3.0,1.0,2.0,2.0,2.0,3.0,6.0,7.0,0.0,2.0,8.0,9.0,10.0,3.00000,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.00000,0.12345,0.0,5.0,4.0,4.0,6.0,3.0,5.0,1.0,5.0,1.0,5.0,1.0,5.0,1.00000,1.00000,1.00000,10.0,10.0,10.0,5.0,6.0,3.0,3.0,2.0,2.0,3.0,3.0,5.0,7.0,2.00000,0.12345,2.00000,0.12345,0.0,7.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,2010.00000,GER,ZZ,2.0,2.0,DE,2.0,DE,1.0,5.0,1.0,8.0,4.0,10.0,5.0,5.0,1.0,5.0,1.0,1.0,1.0,2.0,3.0,4.0,4.0,2.0,2.0,2.0,3.0,1.0,4.0,4.0,8.0,4.0,10.0,10.0,0.0,0.0,0.0,1.0,1.0,2.0,5.0,1.0,2.0,6.0,3.0,10.0,10.0,10.0,5.0,1.0,1.0,5.0,5.0,5.0,1.0,5.0,3.0,5.0,4.0,0.12345,0.12345,0.12345,0.12345,0.12345,0.12345,0.12345,0.12345,0.12345,4.00000,4.00000,4.00000,4.0,0.12345,3.0,4.00000,4.00000,2.0,55.0,0.12345,55.00000,0.12345,1.0,2.0,0.12345,0.12345,0.12345,34.0,1982.0,0.12345,0.12345,0.12345,0.12345,0.12345,0.12345,2.0,0.12345,1.0,2.0,2.0,0.12345,6.00000,6.0,2.0,2.0,1.0,720.0,7.0,21.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.12345,2.0,2.0,2.00000,1.00000,2015.00000,1.0,0.12345,2.00000,1.0,2.0,0.12345,5.0,0.0,25.00000,1.0,40.0,72.00000,2.0,2310.0,2.0,1.0,2.00000,1.00000,3.0,6.0,0.12345,2.0,2.0,0.12345,0.12345,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.12345,0.12345,0.12345,0.12345,0.12345,0.12345,610.0,6.0,1.0,3.0,610.0,6.0,1.0,1.00000,1.0,11070.0,0.12345,2.0,6.0,1.0,2.0,2.0,3.0,5.0,1.0,2.0,4.0,3.0,1.0,2.0,6.0,3.0,6.0,1.0,1.0,1.0,6.0,3.0,AT13,2.0,5.0,12.0,2016.0,7.0,37.0,5.0,12.0,2016.0,8.0,33.0,50.0,0.611677,1.178496,0.370393,0.436506
1,AT,120.0,5.0,120.00000,6.0,6.0,5.0,1.0,1.0,3.0,2.0,3.0,5.0,4.0,3.0,3.0,4.0,6.0,7.0,3.00000,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.

In [164]:
climate_fill.to_csv('./data/climate_fill.csv') #store in CSV

As the filling went a lot faster than expected.. I'll also make a version in which all NaNs are filled with "ZZ" or 0.12345

In [55]:
climate_drop5 = climate.drop(climate_cut.columns[[0,1,2,3,4]], axis=1)
climate_drop5.head()

,cntry,nwspol,netusoft,netustm,ppltrst,pplfair,pplhlp,polintr,psppsgva,actrolga,psppipla,cptppola,trstprl,trstlgl,trstplc,trstplt,trstprt,trstep,trstun,vote,prtvtbat,prtvtcbe,prtvtfch,prtvtdcz,prtvede1,prtvede2,prtvtfee,prtvtdes,prtvtdfi,prtvtcfr,prtvtbgb,prtvtehu,prtvtbie,prtvtcil,prtvtbis,prtvtbit,prtvblt1,prtvblt2,prtvblt3,prtvtfnl,prtvtbno,prtvtdpl,prtvtcpt,prtvtdru,prtvtbse,prtvtesi,contplt,wrkprty,wrkorg,badge,sgnptit,pbldmn,bctprd,pstplonl,clsprty,prtclcat,prtclcbe,prtclfch,prtcldcz,prtclede,prtclfee,prtclees,prtcldfi,prtclefr,prtclbgb,prtclfhu,prtcldie,prtcldil,prtclbis,prtclcit,prtclblt,prtclenl,prtclbno,prtclgpl,prtclept,prtcldru,prtclbse,prtclesi,prtdgcl,lrscale,stflife,stfeco,stfgov,stfdem,stfedu,stfhlth,gincdif,mnrgtjb,freehms,hmsfmlsh,hmsacld,euftf,imsmetn,imdfetn,impcntr,imbgeco,imueclt,imwbcnt,happy,sclmeet,inprdsc,sclact,crmvct,aesfdrk,health,hlthhmp,atchctr,atcherp,rlgblg,rlgdnm,rlgdnbat,rlgdnbe,rlgdnach,rlgdnade,rlgdnafi,rlgdngb,rlgdnhu,rlgdnie,rlgdnis,rlgdnlt,rlgdnnl,rlgdnno,rlgdnapl,rlgdnase,rlgblge,rlgdnme,rlgdebat,rlgdebe,rlgdeach,rlgdeade,rlgdeafi,rlgdegb,rlgdehu,rlgdeie,rlgdeis,rlgdelt,rlgdenl,rlgdeno,rlgdeapl,rlgdease,rlgdgr,rlgatnd,pray,dscrgrp,dscrrce,dscrntn,dscrrlg,dscrlng,dscretn,dscrage,dscrgnd,dscrsex,dscrdsb,dscroth,dscrdk,dscrref,dscrnap,dscrna,ctzcntr,ctzshipc,brncntr,cntbrthc,livecnta,lnghom1,lnghom2,blgetmg,facntr,fbrncntb,mocntr,mbrncntb,gvrfgap,rfgfrpc,rfgbfml,eneffap,rdcenr,cflsenr,elgcoal,elgngas,elghydr,elgnuc,elgsun,elgwind,elgbio,wrpwrct,wrenexp,wrdpimp,wrdpfos,wrntdis,wrinspw,wrtcfl,wrtratc,clmchng,clmthgt1,clmthgt2,ccnthum,ccrdprs,wrclmch,ccgdbd,lkredcc,lklmten,gvsrdcc,ownrdcc,inctxff,sbsrnen,banhhap,dfincac,smdfslv,uemplwk,slvpens,slvuemp,gvslvol,gvslvue,gvcldcr,sbstrec,sbprvpv,sbeqsoc,sbbsntx,sblazy,sblwcoa,imsclbn,uentrjb,lbenent,bennent,admub,ubpay,ubedu,ubunp,ub50pay,ub50edu,ub50unp,ub20pay,ub20edu,ub20unp,ubsppay,ubspedu,ubspunp,bnlwinc,eduunmp,wrkprbf,basinc,eusclbf,eudcnbf,lkuemp,lknemny,vteurmmb,vteumbgb,vteubcmb,hhmmb,gndr,gndr2,gndr3,gndr4,gndr5,gndr6,gndr7,gndr8,gndr9,gndr10,gndr11,gndr12,agea,yrbrn,yrbrn2,yrbrn3,yrbrn4,yrbrn5,yrbrn6,yrbrn7,yrbrn8,yrbrn9,yrbrn10,yrbrn11,yrbrn12,rshipa2,rshipa3,rshipa4,rshipa5,rshipa6,rshipa7,rshipa8,rshipa9,rshipa10,rshipa11,rshipa12,icpart1,rshpsts,rshpsfi,rshpsgb,lvgptnea,dvrcdeva,icpart2,iccohbt,marsts,marstfi,marstgb,maritalb,chldhm,chldhhe,domicil,edulvlb,eisced,edlveat,edlvebe,edlvdch,edlvdcz,edubde1,eduade2,eduade3,edlvdee,edlvges,edlvdfi,edlvdfr,edubgb1,eduagb2,edagegb,edlvdhu,edlvdie,edubil1,eduail2,edlvdis,edlvdit,edlvdlt,edlvenl,edlvdno,edlvepl,edupl2,edlvdpt,edlvdru,edlvdse,edlvesi,eduyrs,pdwrk,edctn,uempla,uempli,dsbld,rtrd,cmsrv,hswrk,dngoth,dngref,dngdk,dngna,icomdng,mainact,mnactic,icpdwrk,crpdwk,pdjobev,pdjobyr,emplrel,emplno,wrkctra,estsz,jbspv,njbspv,wkdcorga,iorgact,wkhct,icwhct,wkhtot,nacer2,tporgwk,isco08,wrkac6m,uemp3m,uemp12m,uemp5yr,mbtru,hincsrca,hinctnta,hincfel,icpart3,edulvlpb,eiscedp,edlvpeat,edlvpebe,edlvpdch,edlvpdcz,edupbde1,edupade2,edupade3,edlvpdee,edlvpfes,edlvpdfi,edlvpdfr,edupbgb1,edupagb2,edagepgb,edlvpdhu,edlvpdie,edupbil1,edupail2,edlvpdis,edlvpdit,edlvpdlt,edlvpenl,edlvpdno,edlvpepl,eduppl2,edlvpdpt,edlvpdru,edlvpdse,edlvpesi,pdwrkp,edctnp,uemplap,uemplip,dsbldp,rtrdp,cmsrvp,hswrkp,dngothp,dngdkp,dngnapp,dngrefp,dngnap,icomdnp,mnactp,icppdwk,crpdwkp,isco08p,emprelp,wkhtotp,edulvlfb,eiscedf,edlvfeat,edlvfebe,edlvfdch,edlvfdcz,edufbde1,edufade2,edufade3,edlvfdee,edlvffes,edlvfdfi,edlvfdfr,edufbgb1,edufagb2,edagefgb,edlvfdhu,edlvfdie,edufbil1,edufail2,edlvfdis,edlvfdit,edlvfdlt,edlvfenl,edlvfdno,edlvfepl,edlvfdpt,edlvfdru,edlvfdse,edlvfesi,emprf14,occf14b,edulvlmb,eiscedm,edlvmeat,edlvmebe,edlvmdch,edlvmdcz,edumbde1,edumade2,edumade3,edlvmdee,edlvmfes,edlvmdfi,edlvmdfr,edumbgb1,edumagb2,edagemgb,edlvmdhu,edlvmdie,edumbil1,edumail2,edlvmdis,edlvmdit,edlvmdlt,edlvmenl,edlvmdno,edlvmepl,edlvmdpt,edlvmdru,edlvmdse,edlvmesi,emprm14,occm14b,atncrse,anctry1,anctry2,ipcrtiv,imprich,ipeqopt,ipshabt,impsaf

In [169]:
#fill NANs in the string columns with "ZZ"
str_cols_5 = climate_drop5.select_dtypes(include='object').columns
str_cols_5

Index(['cntry', 'ctzshipc', 'cntbrthc', 'lnghom1', 'lnghom2', 'fbrncntb',
       'mbrncntb', 'region'],
      dtype='object')

In [170]:
# check if it's save to fill empty values with ZZ 
df_str_cols_5 = climate_drop5.select_dtypes(include='object')

for i in str_cols_5:
    uniques = df_str_cols_5[i].unique() #get unique values per column
    if "ZZ" in uniques: #check if ZZ is in there
        print("ZZ here!")
    else: 
        print("no ZZ!")

no ZZ!
no ZZ!
no ZZ!
no ZZ!
no ZZ!
no ZZ!
no ZZ!
no ZZ!


In [171]:
climate_drop5[str_cols_5] = climate_drop5[str_cols_5].fillna("ZZ") 

In [174]:
climate_drop5.head(10) #check

,cntry,nwspol,netusoft,netustm,ppltrst,pplfair,pplhlp,polintr,psppsgva,actrolga,psppipla,cptppola,trstprl,trstlgl,trstplc,trstplt,trstprt,trstep,trstun,vote,prtvtbat,prtvtcbe,prtvtfch,prtvtdcz,prtvede1,prtvede2,prtvtfee,prtvtdes,prtvtdfi,prtvtcfr,prtvtbgb,prtvtehu,prtvtbie,prtvtcil,prtvtbis,prtvtbit,prtvblt1,prtvblt2,prtvblt3,prtvtfnl,prtvtbno,prtvtdpl,prtvtcpt,prtvtdru,prtvtbse,prtvtesi,contplt,wrkprty,wrkorg,badge,sgnptit,pbldmn,bctprd,pstplonl,clsprty,prtclcat,prtclcbe,prtclfch,prtcldcz,prtclede,prtclfee,prtclees,prtcldfi,prtclefr,prtclbgb,prtclfhu,prtcldie,prtcldil,prtclbis,prtclcit,prtclblt,prtclenl,prtclbno,prtclgpl,prtclept,prtcldru,prtclbse,prtclesi,prtdgcl,lrscale,stflife,stfeco,stfgov,stfdem,stfedu,stfhlth,gincdif,mnrgtjb,freehms,hmsfmlsh,hmsacld,euftf,imsmetn,imdfetn,impcntr,imbgeco,imueclt,imwbcnt,happy,sclmeet,inprdsc,sclact,crmvct,aesfdrk,health,hlthhmp,atchctr,atcherp,rlgblg,rlgdnm,rlgdnbat,rlgdnbe,rlgdnach,rlgdnade,rlgdnafi,rlgdngb,rlgdnhu,rlgdnie,rlgdnis,rlgdnlt,rlgdnnl,rlgdnno,rlgdnapl,rlgdnase,rlgblge,rlgdnme,rlgdebat,rlgdebe,rlgdeach,rlgdeade,rlgdeafi,rlgdegb,rlgdehu,rlgdeie,rlgdeis,rlgdelt,rlgdenl,rlgdeno,rlgdeapl,rlgdease,rlgdgr,rlgatnd,pray,dscrgrp,dscrrce,dscrntn,dscrrlg,dscrlng,dscretn,dscrage,dscrgnd,dscrsex,dscrdsb,dscroth,dscrdk,dscrref,dscrnap,dscrna,ctzcntr,ctzshipc,brncntr,cntbrthc,livecnta,lnghom1,lnghom2,blgetmg,facntr,fbrncntb,mocntr,mbrncntb,gvrfgap,rfgfrpc,rfgbfml,eneffap,rdcenr,cflsenr,elgcoal,elgngas,elghydr,elgnuc,elgsun,elgwind,elgbio,wrpwrct,wrenexp,wrdpimp,wrdpfos,wrntdis,wrinspw,wrtcfl,wrtratc,clmchng,clmthgt1,clmthgt2,ccnthum,ccrdprs,wrclmch,ccgdbd,lkredcc,lklmten,gvsrdcc,ownrdcc,inctxff,sbsrnen,banhhap,dfincac,smdfslv,uemplwk,slvpens,slvuemp,gvslvol,gvslvue,gvcldcr,sbstrec,sbprvpv,sbeqsoc,sbbsntx,sblazy,sblwcoa,imsclbn,uentrjb,lbenent,bennent,admub,ubpay,ubedu,ubunp,ub50pay,ub50edu,ub50unp,ub20pay,ub20edu,ub20unp,ubsppay,ubspedu,ubspunp,bnlwinc,eduunmp,wrkprbf,basinc,eusclbf,eudcnbf,lkuemp,lknemny,vteurmmb,vteumbgb,vteubcmb,hhmmb,gndr,gndr2,gndr3,gndr4,gndr5,gndr6,gndr7,gndr8,gndr9,gndr10,gndr11,gndr12,agea,yrbrn,yrbrn2,yrbrn3,yrbrn4,yrbrn5,yrbrn6,yrbrn7,yrbrn8,yrbrn9,yrbrn10,yrbrn11,yrbrn12,rshipa2,rshipa3,rshipa4,rshipa5,rshipa6,rshipa7,rshipa8,rshipa9,rshipa10,rshipa11,rshipa12,icpart1,rshpsts,rshpsfi,rshpsgb,lvgptnea,dvrcdeva,icpart2,iccohbt,marsts,marstfi,marstgb,maritalb,chldhm,chldhhe,domicil,edulvlb,eisced,edlveat,edlvebe,edlvdch,edlvdcz,edubde1,eduade2,eduade3,edlvdee,edlvges,edlvdfi,edlvdfr,edubgb1,eduagb2,edagegb,edlvdhu,edlvdie,edubil1,eduail2,edlvdis,edlvdit,edlvdlt,edlvenl,edlvdno,edlvepl,edupl2,edlvdpt,edlvdru,edlvdse,edlvesi,eduyrs,pdwrk,edctn,uempla,uempli,dsbld,rtrd,cmsrv,hswrk,dngoth,dngref,dngdk,dngna,icomdng,mainact,mnactic,icpdwrk,crpdwk,pdjobev,pdjobyr,emplrel,emplno,wrkctra,estsz,jbspv,njbspv,wkdcorga,iorgact,wkhct,icwhct,wkhtot,nacer2,tporgwk,isco08,wrkac6m,uemp3m,uemp12m,uemp5yr,mbtru,hincsrca,hinctnta,hincfel,icpart3,edulvlpb,eiscedp,edlvpeat,edlvpebe,edlvpdch,edlvpdcz,edupbde1,edupade2,edupade3,edlvpdee,edlvpfes,edlvpdfi,edlvpdfr,edupbgb1,edupagb2,edagepgb,edlvpdhu,edlvpdie,edupbil1,edupail2,edlvpdis,edlvpdit,edlvpdlt,edlvpenl,edlvpdno,edlvpepl,eduppl2,edlvpdpt,edlvpdru,edlvpdse,edlvpesi,pdwrkp,edctnp,uemplap,uemplip,dsbldp,rtrdp,cmsrvp,hswrkp,dngothp,dngdkp,dngnapp,dngrefp,dngnap,icomdnp,mnactp,icppdwk,crpdwkp,isco08p,emprelp,wkhtotp,edulvlfb,eiscedf,edlvfeat,edlvfebe,edlvfdch,edlvfdcz,edufbde1,edufade2,edufade3,edlvfdee,edlvffes,edlvfdfi,edlvfdfr,edufbgb1,edufagb2,edagefgb,edlvfdhu,edlvfdie,edufbil1,edufail2,edlvfdis,edlvfdit,edlvfdlt,edlvfenl,edlvfdno,edlvfepl,edlvfdpt,edlvfdru,edlvfdse,edlvfesi,emprf14,occf14b,edulvlmb,eiscedm,edlvmeat,edlvmebe,edlvmdch,edlvmdcz,edumbde1,edumade2,edumade3,edlvmdee,edlvmfes,edlvmdfi,edlvmdfr,edumbgb1,edumagb2,edagemgb,edlvmdhu,edlvmdie,edumbil1,edumail2,edlvmdis,edlvmdit,edlvmdlt,edlvmenl,edlvmdno,edlvmepl,edlvmdpt,edlvmdru,edlvmdse,edlvmesi,emprm14,occm14b,atncrse,anctry1,anctry2,ipcrtiv,imprich,ipeqopt,ipshabt,impsaf

In [175]:
#fill the NaNs in numerical columns with 0.12345
climate_full_fill = climate_drop5.fillna(0.12345)

In [176]:
#check:
climate_full_fill.head()

,cntry,nwspol,netusoft,netustm,ppltrst,pplfair,pplhlp,polintr,psppsgva,actrolga,psppipla,cptppola,trstprl,trstlgl,trstplc,trstplt,trstprt,trstep,trstun,vote,prtvtbat,prtvtcbe,prtvtfch,prtvtdcz,prtvede1,prtvede2,prtvtfee,prtvtdes,prtvtdfi,prtvtcfr,prtvtbgb,prtvtehu,prtvtbie,prtvtcil,prtvtbis,prtvtbit,prtvblt1,prtvblt2,prtvblt3,prtvtfnl,prtvtbno,prtvtdpl,prtvtcpt,prtvtdru,prtvtbse,prtvtesi,contplt,wrkprty,wrkorg,badge,sgnptit,pbldmn,bctprd,pstplonl,clsprty,prtclcat,prtclcbe,prtclfch,prtcldcz,prtclede,prtclfee,prtclees,prtcldfi,prtclefr,prtclbgb,prtclfhu,prtcldie,prtcldil,prtclbis,prtclcit,prtclblt,prtclenl,prtclbno,prtclgpl,prtclept,prtcldru,prtclbse,prtclesi,prtdgcl,lrscale,stflife,stfeco,stfgov,stfdem,stfedu,stfhlth,gincdif,mnrgtjb,freehms,hmsfmlsh,hmsacld,euftf,imsmetn,imdfetn,impcntr,imbgeco,imueclt,imwbcnt,happy,sclmeet,inprdsc,sclact,crmvct,aesfdrk,health,hlthhmp,atchctr,atcherp,rlgblg,rlgdnm,rlgdnbat,rlgdnbe,rlgdnach,rlgdnade,rlgdnafi,rlgdngb,rlgdnhu,rlgdnie,rlgdnis,rlgdnlt,rlgdnnl,rlgdnno,rlgdnapl,rlgdnase,rlgblge,rlgdnme,rlgdebat,rlgdebe,rlgdeach,rlgdeade,rlgdeafi,rlgdegb,rlgdehu,rlgdeie,rlgdeis,rlgdelt,rlgdenl,rlgdeno,rlgdeapl,rlgdease,rlgdgr,rlgatnd,pray,dscrgrp,dscrrce,dscrntn,dscrrlg,dscrlng,dscretn,dscrage,dscrgnd,dscrsex,dscrdsb,dscroth,dscrdk,dscrref,dscrnap,dscrna,ctzcntr,ctzshipc,brncntr,cntbrthc,livecnta,lnghom1,lnghom2,blgetmg,facntr,fbrncntb,mocntr,mbrncntb,gvrfgap,rfgfrpc,rfgbfml,eneffap,rdcenr,cflsenr,elgcoal,elgngas,elghydr,elgnuc,elgsun,elgwind,elgbio,wrpwrct,wrenexp,wrdpimp,wrdpfos,wrntdis,wrinspw,wrtcfl,wrtratc,clmchng,clmthgt1,clmthgt2,ccnthum,ccrdprs,wrclmch,ccgdbd,lkredcc,lklmten,gvsrdcc,ownrdcc,inctxff,sbsrnen,banhhap,dfincac,smdfslv,uemplwk,slvpens,slvuemp,gvslvol,gvslvue,gvcldcr,sbstrec,sbprvpv,sbeqsoc,sbbsntx,sblazy,sblwcoa,imsclbn,uentrjb,lbenent,bennent,admub,ubpay,ubedu,ubunp,ub50pay,ub50edu,ub50unp,ub20pay,ub20edu,ub20unp,ubsppay,ubspedu,ubspunp,bnlwinc,eduunmp,wrkprbf,basinc,eusclbf,eudcnbf,lkuemp,lknemny,vteurmmb,vteumbgb,vteubcmb,hhmmb,gndr,gndr2,gndr3,gndr4,gndr5,gndr6,gndr7,gndr8,gndr9,gndr10,gndr11,gndr12,agea,yrbrn,yrbrn2,yrbrn3,yrbrn4,yrbrn5,yrbrn6,yrbrn7,yrbrn8,yrbrn9,yrbrn10,yrbrn11,yrbrn12,rshipa2,rshipa3,rshipa4,rshipa5,rshipa6,rshipa7,rshipa8,rshipa9,rshipa10,rshipa11,rshipa12,icpart1,rshpsts,rshpsfi,rshpsgb,lvgptnea,dvrcdeva,icpart2,iccohbt,marsts,marstfi,marstgb,maritalb,chldhm,chldhhe,domicil,edulvlb,eisced,edlveat,edlvebe,edlvdch,edlvdcz,edubde1,eduade2,eduade3,edlvdee,edlvges,edlvdfi,edlvdfr,edubgb1,eduagb2,edagegb,edlvdhu,edlvdie,edubil1,eduail2,edlvdis,edlvdit,edlvdlt,edlvenl,edlvdno,edlvepl,edupl2,edlvdpt,edlvdru,edlvdse,edlvesi,eduyrs,pdwrk,edctn,uempla,uempli,dsbld,rtrd,cmsrv,hswrk,dngoth,dngref,dngdk,dngna,icomdng,mainact,mnactic,icpdwrk,crpdwk,pdjobev,pdjobyr,emplrel,emplno,wrkctra,estsz,jbspv,njbspv,wkdcorga,iorgact,wkhct,icwhct,wkhtot,nacer2,tporgwk,isco08,wrkac6m,uemp3m,uemp12m,uemp5yr,mbtru,hincsrca,hinctnta,hincfel,icpart3,edulvlpb,eiscedp,edlvpeat,edlvpebe,edlvpdch,edlvpdcz,edupbde1,edupade2,edupade3,edlvpdee,edlvpfes,edlvpdfi,edlvpdfr,edupbgb1,edupagb2,edagepgb,edlvpdhu,edlvpdie,edupbil1,edupail2,edlvpdis,edlvpdit,edlvpdlt,edlvpenl,edlvpdno,edlvpepl,eduppl2,edlvpdpt,edlvpdru,edlvpdse,edlvpesi,pdwrkp,edctnp,uemplap,uemplip,dsbldp,rtrdp,cmsrvp,hswrkp,dngothp,dngdkp,dngnapp,dngrefp,dngnap,icomdnp,mnactp,icppdwk,crpdwkp,isco08p,emprelp,wkhtotp,edulvlfb,eiscedf,edlvfeat,edlvfebe,edlvfdch,edlvfdcz,edufbde1,edufade2,edufade3,edlvfdee,edlvffes,edlvfdfi,edlvfdfr,edufbgb1,edufagb2,edagefgb,edlvfdhu,edlvfdie,edufbil1,edufail2,edlvfdis,edlvfdit,edlvfdlt,edlvfenl,edlvfdno,edlvfepl,edlvfdpt,edlvfdru,edlvfdse,edlvfesi,emprf14,occf14b,edulvlmb,eiscedm,edlvmeat,edlvmebe,edlvmdch,edlvmdcz,edumbde1,edumade2,edumade3,edlvmdee,edlvmfes,edlvmdfi,edlvmdfr,edumbgb1,edumagb2,edagemgb,edlvmdhu,edlvmdie,edumbil1,edumail2,edlvmdis,edlvmdit,edlvmdlt,edlvmenl,edlvmdno,edlvmepl,edlvmdpt,edlvmdru,edlvmdse,edlvmesi,emprm14,occm14b,atncrse,anctry1,anctry2,ipcrtiv,imprich,ipeqopt,ipshabt,impsaf

In [177]:
climate_full_fill.to_csv('./data/climate_full_fill.csv') #store in CSV